## Основная задача: 
Собрать из открытых источников что-то похожее на датасет предсказания банкротства (https://www.kaggle.com/datasets/fedesoriano/company-bankruptcy-prediction)

- Большая часть информации - xml-файлы и несколько справочников csv и xlsx - отсюда: https://www.nalog.gov.ru/opendata/

- Важно как можно раньше начать собирать данные финансовых и бухгалтерских отчётов пауком с сайта https://bo.nalog.ru/ - так как это наиболее длительная часть проекта

- Ссылка на страницу с отчётами осуществляется по id карточки организации, который можно узнать, сделав запрос по номеру инн юр. лица (однозначно определяет организацию)

- Далее, отчёты есть только для юридических лиц (т.е. необходимо знать ИННЮЛ), тех из них, которые обязаны предоставлять открытую информацию (ИНН физ. лиц не нужны)

- Т.е. необходим список иннюл - его можно достать из реестра МСП (малых и средних предприятий), чтобы первым делом начать собирать ту информацию, к которой нет прямого доступа.

- xml достаточно громоздкие, но к ним есть описание структуры и данные достаточно чистые.

In [89]:
import os
import pickle
import requests
import psycopg2
import pandas as pd
import numpy as np

from lxml import etree
from psycopg2 import sql
from pymongo import MongoClient

### 1. Сбор данных из разных источников
#### 1.1. Паук для сбора данных

In [4]:
# путь к директории с файлами реестра мсп, около 20 гб, около 7 тыс. файлов
path = 'raw_data/reestr_ms/data-10032023-structure-10032022'
files = [path + '/' + file for file in os.listdir(path)]
len(files)

6794

In [5]:
#  пусть будет 7 отрезков по 1000 файлов (в последнем - 794)
k = 1000
cut_files = [files[i: i + k] for i in range(0, 7000, 1000) ]

- получилось 7 дампов pickle, примерно по 350 тыс ИННЮЛ в каждом списке

- общая длина - около 2.3 млн

- здесь обработаем и сразу запишем для удобства в несколько файлов (оказалось, что для паука можно было делать файлы ещё короче)

In [ ]:
# %%time

# for j, files in enumerate(cut_files):
    
#     inn = []
#     for i, file in enumerate(files):
        
#         if i % 100 == 0: print(i)
        
#         tree = etree.parse(file)
#         root = tree.getroot()
#         
#         for doc in root.findall('Документ/ОргВклМСП'):
#             inn.append(doc.attrib['ИННЮЛ'])
            
    
#     with open(f'parsed_data/inn/inn_{j}.pickle', 'wb') as f:
#         pickle.dump(inn, f)  

Дальше более подробно изучила, какая информация есть на страничках с отчётами разных организаций:

- бухгалтерский баланс за 3 года и финансовые результаты за 2 года есть для всех типов (если не искать информацию целенаправленно по какому-то критерию (типу организации), собственности и т.п. - то можно собрать именно их)

Но оказалось, что страницы генерируются динамически - простой запрос практически ничего не возвращает:

- для загрузки страницы отправляется множество get-запросов, каждый из которых возвращает json для наполнения контентом.
При этом, только для сбора основной информации (карточка компании, фин. результаты и бух. отчет) нужно минимум 4 запроса.

Итоговая логика переходов:
1. карточка организации по инн получим id организации (id_org) на сайте
        f'https://bo.nalog.ru/nbo/organizations/search?query={ИНН}&page=0'
2. переход на следующую страницу - по полученному id_org, соберём основные данные о организации
        f'https://bo.nalog.ru/nbo/organizations/{id_org}'
3. снова переход по id_org - получим id финансовых и бухгалтерских отчётов (id_doc)
        f'https://bo.nalog.ru/nbo/organizations/{id_org}/bfo/'
4. переход по id_doc - забираем сами отчёты.
        f'https://bo.nalog.ru/nbo/bfo/{id_doc}/details'

Код паука - см. bfoparser

- паук обрабатывает кусок инн в 50000 * 4 перехода = около 200 тыс запросов (меньше, так как не для всех инн выдаётся информация), с DOWNLOAD_DELAY = 0.2 и CONCURRENT_REQUESTS = 32 (примерно 244 запросов и 60 записей в минуту) это займёт около 13 часов, на выходе примерно 40 тыс. записей.
- запись идёт в MongoDB (проще и быстрее, чем писать в файл).

#### 1.2. Xml-файлы

- складываю в postgresql:
    - удобно хранить - сущности со связями (пока просто несвязанные таблицы, основные связи будут по ИНН)
    - удобно делать пакетную вставку при обработке пофайлово


#### -  xml с реестром мсп (малых и средних предприятий)

https://www.nalog.gov.ru/opendata/7707329152-rsmp/

забираем:

    - ССЧР (Сведения о среднесписочной численности работников)
    - СведСоцПред (Сведения о том, что юридическое лицо / индивидуальный предприниматель является социальным предприятием, 1,2), 
    - ПризНовМСП (Признак сведений о вновь созданном юридическом лице / вновь зарегистрированном индивидуальном предпринимателе, 1,2), 
    - КатСубМСП (Категория субъекта МСП - ?, 1,2,3), 
    - ВидСубМСП (Вид субъекта МСП - ?, 1,2), 
    - ДатаВклМСП
    - ИННЮЛ - будет ключом
    - КодОКВЭД
    - если есть СвЛиценз (сведения о лицензиях) - счётчик, если несколько: считаем вложенные объекты НаимЛицВД (наименование лицензии)
    - СвПрод (Сведения о производимой субъектом МСП продукции) - если есть, мб несколько объектов - собираем в список атрибут КодПрод (Код вида продукции) (это справочник ОКПД2?), 
    - если есть хотя бы 1 атрибут ПрОтнПрод == 1 (признак отнесения продукциик инновационной, высокотехнологичной) - можно взять как бинарный признак
    - объект СвКонтр - мб несколько (Сведения о наличии у субъекта МСП в предшествующем календарном году контрактов, заключенных в соответствии с Федеральным законом от 5 апреля 2013 года №44-ФЗ) - для обеспечения государственных и муниципальных нужд: взять число контрактов
    - СвДог (Сведения о наличии у субъекта МСП в предшествующем календарном году договоров, заключенных в соответствии с Федеральным законом от 18 июля 2011 года №223-ФЗ) - О закупках товаров, работ, услуг: мб несколько, взять число

In [11]:
file = "../raw_data/reestr_ms/data-10032023-structure-10032022/VO_RRMSPSV_0000_9965_20230310_00a62ea7-b905-4787-aec9-3d44f97cdb1f.xml"
tree = etree.parse(file)
root = tree.getroot()
docs = root.findall("Документ[ОргВклМСП]")
doc = docs[1]

In [12]:
print(etree.tostring(doc, encoding = "unicode", pretty_print=True))

<Документ ИдДок="308515e2-59fe-4000-b750-8afdd4f6e095" ДатаСост="10.03.2023" ДатаВклМСП="10.09.2019" ВидСубМСП="1" КатСубМСП="1" ПризНовМСП="2" СведСоцПред="2" ССЧР="6">
  <ОргВклМСП НаимОрг="ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot;ТЕКЛАЙН&quot;" НаимОргСокр="ООО &quot;ТЕКЛАЙН&quot;" ИННЮЛ="4702020559" ОГРН="1194704019894"/>
  <СведМН КодРегион="47">
    <Регион Тип="ОБЛАСТЬ" Наим="ЛЕНИНГРАДСКАЯ"/>
    <Район Тип="РАЙОН" Наим="ВОЛХОВСКИЙ"/>
    <Город Тип="ГОРОД" Наим="ВОЛХОВ"/>
  </СведМН>
  <СвОКВЭД>
    <СвОКВЭДОсн КодОКВЭД="42.11" НаимОКВЭД="Строительство автомобильных дорог и автомагистралей" ВерсОКВЭД="2014"/>
    <СвОКВЭДДоп КодОКВЭД="77.32" НаимОКВЭД="Аренда и лизинг строительных машин и оборудования" ВерсОКВЭД="2014"/>
    <СвОКВЭДДоп КодОКВЭД="52.29" НаимОКВЭД="Деятельность вспомогательная прочая, связанная с перевозками" ВерсОКВЭД="2014"/>
    <СвОКВЭДДоп КодОКВЭД="46.73.6" НаимОКВЭД="Торговля оптовая прочими строительными материалами и изделиями" ВерсОКВЭД="2014"/>
 

In [2]:
# небольшая очистка
def clean(val):
    if isinstance(val, str):
        val = val.strip()
#         if val.isdigit():
#             return int(val)
    return val

- неожиданная проблема возникла после очистки инн - неуникальные инн (в итоге - всего 2): проще разрешить неуникальность и потом вычислить такие инн, чем проверять каждый инн до формирования пакета
- поэтому делаю временный primary key для таблицы, а после вставки - вычищаю повторы и изменяю ключ

In [3]:
# таблица reestr_msp
# Критерии среднего предприятия - не более 1500 человек
# формат даты распознаёт и приводит автоматически
reestr_sql = 'CREATE TABLE reestr_msp (\
temp_id bigserial PRIMARY KEY, \
inn int8, \
n_employees int2, \
is_soc_enterprise int2, \
newly_establ int2, \
subj_cat int2, \
subj_form int2, \
inclusion_date date, \
n_okovd_add int2, \
okovd_add_vers varchar (8), \
okovd_add_code text [], \
n_licenses int2, \
n_products int2, \
products_code text [], \
is_high_tech int2, \
n_contracts int2, \
n_agreements int2 \
);'

In [ ]:
# обёртка для транзакции (вместо commit())
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(reestr_sql)

connect.close()

In [4]:
# функция - для получения одного val
def get_val(doc):
    """
    Принимает один документ
    Возвращает необходимые поля, упакованые в кортеж
    """
    val = []
    # это ключ
    val.append(clean(doc.find('ОргВклМСП').attrib['ИННЮЛ']))
    # некоторых атрибутов может не быть
    try:
        val.append(clean(doc.attrib['ССЧР']))
    except KeyError:
        val.append(None)
    try:
        # является ли соц. предприятием
        val.append(1 if doc.attrib['СведСоцПред'] == '1' else 0)
    except KeyError:
        val.append(None)        
    try:
        # вновь создано?
        val.append(1 if doc.attrib['ПризНовМСП'] == '1' else 0 )
    except KeyError:
        val.append(None)
    try:    
        # категория организации
        val.append(clean(doc.attrib['КатСубМСП']))  # 1, 2, 3, не знаю, что - эти двое совпадают?
    except KeyError:
        val.append(None)    
    try:    
        # форма
        val.append(clean(doc.attrib['ВидСубМСП'])) # 1, 2, 3, не знаю, что
    except KeyError:
        val.append(None)
    try:
        # дата включения в реестр
        val.append(doc.attrib['ДатаВклМСП'])
    except KeyError:
        val.append(None)

    # а здесь - по видам деятельности, код основного - из mongo, а из дополнительных - если есть:
    # если ничего нет, ошибки не будет
    add_occupations = doc.findall('СвОКВЭД/СвОКВЭДДоп')
    val.append(len(add_occupations))  # число доп видов деятельности
    if add_occupations:
        val.append(add_occupations[0].attrib['ВерсОКВЭД'])  # 2014 = okovd2   
        val.append([el.attrib['КодОКВЭД'] for el in add_occupations])  # список кодов
    else:
        # если нет добавочной деятельности => нет и соотв версии
        val.append(None)
        # и список доп.кодов пустой
        val.append([])

    # дальше - редкие признаки
    # сведения о лицензиях - число лицензий
    val.append(len(doc.findall('СвЛиценз')))

    # сведения о производимой продкукции
    # число типов продукции, список их кодов, если есть хотя бы 1 ПрОтнПрод == 1 - бинарный признак: 
    # производит инновац/высокотехнологич продукцию
    products = doc.findall('СвПрод')
    val.append(len(products))
    if products:
        # список кодов
        val.append([el.attrib['КодПрод'] for el in products])
        # высокотехнологичный? 1 - да, 2 - нет, аналогично меняю на 1-0
        val.append(1 if '1' in [el.attrib['ПрОтнПрод'] for el in products] else 0)
    else:
        val.append([])
        val.append(0)

    # сведения о контрактах -  ФЗ "О контрактной системе в сфере закупок товаров, работ, услуг 
    # для обеспечения государственных и муниципальных нужд"
    # число контрактов
    # мб - объеденить с договорами?
    val.append(len(doc.findall('СвКонтр')))

    # сведения о договорах - ФЗ "О закупках товаров, работ, услуг отдельными видами юридических лиц" 
    # число договоров
    val.append(len(doc.findall('СвДог')))

    return tuple(val)

In [ ]:
path = '../raw_data/reestr_ms/data-10032023-structure-10032022'
xpath = "Документ[ОргВклМСП]"
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
table_name = 'reestr_msp'
table_columns = ['inn', 'n_employees', 'is_soc_enterprise', 'newly_establ', 'subj_cat', 
                 'subj_form', 'inclusion_date', 'n_okovd_add', 'okovd_add_vers', 'okovd_add_code', 
                 'n_licenses', 'n_products', 'products_code', 'is_high_tech', 'n_contracts', 
                 'n_agreements']

In [5]:
# функция для итерирования по файлам, сбора документов и вставки в таблицу
# тоже разбить на несколько небольших функций?
def insert_to_pgsql(path, xpath, connect, table_name, table_columns):
    files = [path + '/' + file for file in os.listdir(path)]
    for file in files:
        tree = etree.parse(file)
        root = tree.getroot()
        docs = root.findall(xpath)
        
#         print(len(docs), file)
        # если список не пуст:
        if docs:
            values = []
            for doc in docs:
                val = get_val(doc)
                values.append(val)
            
            # транзакции не равномерны - от 1-2 до 900 значений
            # и стоит ли каждый раз закрывать курсор?
            with connect:
                with connect.cursor() as cur:
                    insert = (
                         sql.SQL(
                          'INSERT INTO {} ({}) VALUES {}')
                          .format(sql.Identifier(table_name), 
                                  sql.SQL(',').join(map(sql.Identifier, table_columns)), 
                                  sql.SQL(',').join(map(sql.Literal, values))
                         )
                        )
                    cur.execute(insert)

    connect.close()

- Wall time: 19min 14s, count 2 332 303

In [ ]:
# %%time
# insert_to_pgsql(path, xpath, connect, table_name, table_columns)

- меняем ключ

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [ ]:
# одинаковые инн
with connect:
    with connect.cursor() as cur:
        select = 'SELECT inn, count(inn) FROM reestr_msp rm GROUP BY inn HAVING count(inn) > 1;'
        cur.execute(select)
        print(cur.fetchall())

    [(6371005166, 2), (6319000782, 2)]

In [ ]:
# удаляем все 4 записи
with connect:
    with connect.cursor() as cur:
        delete = 'DELETE FROM reestr_msp WHERE inn in (6371005166, 6319000782);'
        cur.execute(delete)

In [ ]:
# и меняем таблицу - удаляем старый первичный ключ, добавляем новый
# это долго - так как проверяет все inn на условие...
with connect:
    with connect.cursor() as cur:
        alter1 = 'ALTER TABLE reestr_msp DROP COLUMN temp_id;'
        alter2 = 'ALTER TABLE reestr_msp ADD PRIMARY KEY (inn);'
        cur.execute(alter1)
        cur.execute(alter2)

In [ ]:
connect.close()

#### - Сведения о суммах недоимки и задолженности по пеням и штрафам

https://www.nalog.gov.ru/opendata/7707329152-debtam/

- Сведения указываются по состоянию на 31 декабря года, предшествующего году их размещения на сайте ФНС России, при наличии такой недоимки и задолженности по пеням и штрафам по состоянию на 1 октября года размещения указанных сведений на сайте ФНС России

- что-то из этого может быть целью

- здесь только юр лица

Собираем:
    
    - ДатаСост - Дата, по состоянию на которую, подготовлены данные для публикации
    
    - ИННЮЛ - id, очистка и уникальность
    
    - несколько СведНедоим суммируем по атрибутам: 
      
        - СумШтраф - конкретная сумма, ненадлежащее исполнение
        - СумПени - за дни просрочки
        - СумНедНалог - сама недоимка
        
        Сами СведНедоим разбиты по имени налога ("Страховые и другие взносы на обязательное пенсионное страхование, зачисляемые в Пенсионный фонд Российской Федерации", "НЕНАЛОГОВЫЕ ДОХОДЫ, администрируемые налоговыми органами"), при этом какой-либо код отсутствует, а искать по длинной строке с возможными опечатками стоит только, если нужно что-то конкретное.

In [13]:
file = '../raw_data\debt\VO_OT6\VO_OTKRDAN6_9965_9965_20221220_000cc2e4-2682-40ab-9982-98870d1423d9.xml'
tree = etree.parse(file)
root = tree.getroot()
docs = root.findall("Документ")
doc = docs[0]

In [14]:
print(etree.tostring(doc, encoding = "unicode", pretty_print=True))

<Документ ИдДок="30141970-7ee6-44c9-8bb6-5db64b0576c1" ДатаДок="20.12.2022" ДатаСост="31.12.2021">
  <СведНП НаимОрг="ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot;СУББОТНИК&quot;" ИННЮЛ="7430020629"/>
  <СведНедоим НаимНалог="Страховые взносы на обязательное медицинское страхование работающего населения, зачисляемые в бюджет Федерального фонда обязательного медицинского страхования" СумНедНалог="0.00" СумПени="56.14" СумШтраф="0.00" ОбщСумНедоим="56.14"/>
  <СведНедоим НаимНалог="НЕНАЛОГОВЫЕ ДОХОДЫ, администрируемые налоговыми органами" СумНедНалог="0.00" СумПени="0.00" СумШтраф="500.00" ОбщСумНедоим="500.00"/>
  <СведНедоим НаимНалог="Налог, взимаемый в связи с  применением упрощенной  системы налогообложения" СумНедНалог="47026.42" СумПени="29529.77" СумШтраф="16360.00" ОбщСумНедоим="92916.19"/>
  <СведНедоим НаимНалог="Страховые и другие взносы на обязательное пенсионное страхование, зачисляемые в Пенсионный фонд Российской Федерации" СумНедНалог="0.00" СумПени="242.16" СумШтраф="

In [ ]:
# для обработки сумм
def to_float(val):
    if isinstance(val, str):
        val = val.strip().replace(',', '.')      
        try:
            return float(val)        
        except ValueError:                                           
            return 0 # val  # чтобы не терять другие значения
    return val

In [ ]:
def get_val(doc):
    """
    Принимает один документ
    Возвращает необходимые поля, упакованые в кортеж
    """
    val = []
    # инн = ключ
    val.append(clean(doc.find('СведНП').attrib['ИННЮЛ']))
    # дата, по состоянию на которую есть данные
    val.append(clean(doc.attrib['ДатаСост']))
    # общая сумма по видам налогов: сама недоимка, пени (за дни просрочки), 
    # штраф (конкретная сумма, ненадлежащее исполнение)
    all_debts = doc.findall('СведНедоим')
    # недоимка
    arrears = 0
    # пени
    penalties = 0
    # штраф
    fines = 0
    for debt in all_debts:
        arrears += to_float(debt.attrib['СумНедНалог'])
        penalties += to_float(debt.attrib['СумПени'])
        fines += to_float(debt.attrib['СумШтраф'])
    val.extend([arrears, penalties, fines])
    
    return tuple(val)

In [ ]:
# таблица
debt_sql = 'CREATE TABLE debt (\
inn int8 PRIMARY KEY, \
status_date date, \
arrears money, \
penalties money, \
fines money \
);'

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(debt_sql)

connect.close()

In [ ]:
path = '../raw_data/debt/VO_OT6'
xpath = "Документ"  # содержит только ИННЮЛ
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
table_name = 'debt'
table_columns = ['inn', 'status_date', 'arrears', 'penalties', 'fines']

In [ ]:
# %%time
# insert_to_pgsql(path, xpath, connect, table_name, table_columns)

- Wall time: 5min 50s, count 1 080 232

#### - реестр субъектов малого и среднего предпринимательства – получателей поддержки

https://www.nalog.gov.ru/opendata/7707329152-rsmppp/

- опять же, много видов поддержки, возьмём только финансовую

- Документ (у которого есть элемент СвЮЛ)
    - ИННЮЛ
    - ИнфНаруш - 1 или 2 - Информация о наличии нарушения порядка и условий оказания поддержки
    - ДатаПрин - дата принятия решения о предоставлении поддержки
    - ФормПод - форма поддержки

        отбираем тех, у кого атрибут КодФорм="0100" - это финансовая поддержка
        - на этом же уровне - элемент ВидПод - 
            - субсидии и гранты, КодВид="0103" ед 1, видимо деньги
            - финансирование на возвратной основе КодВид="0104" ед поддержки 4
            - гарантии и поручительство ="0102"
        - РазмПод имеет значения в зависимости от атрибута ЕдПод - Единица измерения поддержки, значение 1 (деньги), 2, 3 (видимо, число консультаций), 4 (не знаю, что, процент?), 5 (мероприятия?)

- может быть несколько поддержек для 1 организации одного вида или разных видов
- так как не знаю, что ищу, можно сделать строка = конкретный акт поддержки (а не организация)
- тем более, что фактор времени (дата принятия решения о поддержке) - пока не знаю, какой период нужен

In [9]:
file = "../raw_data/support_ms/data/VO_SVMSP_0000_9965_20230315_00a60c8b-cb46-42f4-bb16-beee983cf34d.xml"
tree = etree.parse(file)
root = tree.getroot()
docs = root.findall("Документ[СвЮЛ]")
doc = docs[1]

In [10]:
print(etree.tostring(doc, encoding = "unicode", pretty_print=True))

<Документ ИдДок="264abcc9-c501-4845-89df-3d77cdbb15a6" ДатаСост="15.03.2023">
  <СвЮЛ ИННЮЛ="3810047203" НаимОрг="ООО &quot;БАРГУЗИН&quot;"/>
  <СвПредПод ВидПП="1" НаимОрг="ФЕДЕРАЛЬНАЯ НАЛОГОВАЯ СЛУЖБА" ИННЮЛ="7707329152" КатСуб="1" СрокПод="19.06.2020" ДатаПрин="19.06.2020" ИнфНаруш="2">
    <ФормПод КодФорм="0100" НаимФорм="Финансовая поддержка"/>
    <ВидПод КодВид="0103" НаимВид="Предоставление субсидий и грантов"/>
    <РазмПод РазмПод="48520.00" ЕдПод="1"/>
  </СвПредПод>
  <СвПредПод ВидПП="1" НаимОрг="ФЕДЕРАЛЬНАЯ НАЛОГОВАЯ СЛУЖБА" ИННЮЛ="7707329152" КатСуб="1" СрокПод="19.05.2020" ДатаПрин="19.05.2020" ИнфНаруш="2">
    <ФормПод КодФорм="0100" НаимФорм="Финансовая поддержка"/>
    <ВидПод КодВид="0103" НаимВид="Предоставление субсидий и грантов"/>
    <РазмПод РазмПод="48520.00" ЕдПод="1"/>
  </СвПредПод>
  <СвПредПод ВидПП="1" НаимОрг="ФЕДЕРАЛЬНАЯ НАЛОГОВАЯ СЛУЖБА" ИННЮЛ="7707329152" КатСуб="1" СрокПод="25.11.2021" ДатаПрин="25.11.2021" ИнфНаруш="2">
    <ФормПод КодФорм="010

In [15]:
def get_values(doc):
    """
    Принимает документ
    Возвращает несколько строк для вставки (список кортежей) или ни одной строки
    """   
    values = []
    # общие данные для строк одного документа
    inn = clean(doc.find('СвЮЛ').attrib['ИННЮЛ'])
    # вся финансовая поддержка
    supports = doc.findall('СвПредПод')

    for support in supports:
        # если финансовая поддержка - забираем
        if support.find('ФормПод').attrib['КодФорм'] == "0100":
            val = []
            val.append(inn)
            # дата принятия решения о предоставлении поддержки
            val.append(support.attrib['ДатаПрин'])
            # есть ли нарушение? 1 - да, 0 - нет
            val.append(1 if support.attrib['ИнфНаруш'] == "1" else 0)
            # код формы: 0102 - гарантии и поручительство, 0103 - субсидии и гранты, 
            # 0104 - возвратная основа
            val.append(clean(support.find('ВидПод').attrib['КодВид']))
            # размер
            val.append(to_float(support.find('РазмПод').attrib['РазмПод']))
            # единица: 1 (деньги), 2, 3 (число консультаций?), 4 (процент?), 5 (мероприятия?)
            # 1 для 0102 и 0103, 4 для 0104
            val.append(clean(support.find('РазмПод').attrib['ЕдПод']))

            values.append(tuple(val))
    
    return values

In [16]:
# и своя ф-я для вставки, ещё более громоздкая!
def inserts_to_pgsql(path, xpath, connect, table_name, table_columns):
    files = [path + '/' + file for file in os.listdir(path)]
    for file in files:
        tree = etree.parse(file)
        root = tree.getroot()
        docs = root.findall(xpath)
        
#         print(len(docs), file)
        # если список не пуст:
        if docs:
            values = []
            for doc in docs:
                val_list = get_values(doc)
                # если не пуст
                if val_list:
                    values.extend(val_list)

            if values:
                with connect:
                    with connect.cursor() as cur:
                        insert = (
                             sql.SQL(
                              'INSERT INTO {} ({}) VALUES {}')
                              .format(sql.Identifier(table_name), 
                                      sql.SQL(',').join(map(sql.Identifier, table_columns)), 
                                      sql.SQL(',').join(map(sql.Literal, values))
                             )
                            )
                        cur.execute(insert)

    connect.close()

In [17]:
support_sql = 'CREATE TABLE support_ms (\
id bigserial PRIMARY KEY, \
inn int8, \
support_date date, \
is_breach int2, \
form_code char (4), \
support_volume money, \
support_unit int2 \
);'

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(support_sql)

connect.close()

In [ ]:
path = '../raw_data/support_ms/data'
xpath = "Документ[СвЮЛ]"
connect = psycopg2.connect(dbname="fns_data", user="user", password='new_password', host="localhost", port=5432)
table_name = 'support_ms'
table_columns = ['inn', 'support_date', 'is_breach', 'form_code', 'support_volume', 'support_unit']

In [ ]:
# %%time
# inserts_to_pgsql(path, xpath, connect, table_name, table_columns)

- Wall time: 10min 23s
- Count 1 170 650, count(DISTINCT inn) - 381 279
- уникальные даты поддержки - с 2019 по 2023

#### - Сведения о налоговых правонарушениях и мерах ответственности за их совершение

https://www.nalog.gov.ru/opendata/7707329152-taxoffence/

Сведения указываются в отношении налоговых правонарушений, решения о привлечении к ответственности за совершение которых вступили в силу в период с 1 января по 31 декабря года, предшествующего году размещения указанных сведений на сайте ФНС России, при неуплате штрафа в срок до 1 октября года размещения таких сведений на сайте ФНС России

- все ИННЮЛ и по 1 нарушению для каждого

забираем:

    - ДатаСост - Дата, по состоянию на которую, подготовлены данные для публикации
    - ИННЮЛ
    - СумШтраф - сумма штрафов

In [ ]:
def get_val(doc):
    """
    Принимает один документ
    Возвращает необходимые поля, упакованые в кортеж
    """
    val = []
    # инн = ключ
    val.append(clean(doc.find('СведНП').attrib['ИННЮЛ']))
    # дата, по состоянию на которую есть данные
    val.append(clean(doc.attrib['ДатаСост']))
    # сумма штрафа
    val.append(to_float(doc.find('СведНаруш').attrib['СумШтраф']))
    
    return tuple(val)

- столбцы таблицы:

        inn
        offense_date - дата состояния, а не нарушения!
        fine_amount

In [ ]:
tax_offense_sql = 'CREATE TABLE tax_offense (\
inn text PRIMARY KEY, \
offense_date date, \
fine_amount decimal \
);'

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(tax_offense_sql)

connect.close()

In [ ]:
path = 'raw_data/tax_offense/data'
xpath = "Документ"
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
table_name = 'tax_offense'
table_columns = ['inn', 'offense_date', 'fine_amount']

- Wall time: 4min 24s
- COUNT(*) 129 827
- DISTINCT offense_date 2021-12-31 - те одна дата

In [ ]:
# %%time
# insert_to_pgsql(path, xpath, connect, table_name, table_columns)

In [ ]:
connect.close()

#### 1.3. дополнительные данные

#### - Юридические лица, в состав исполнительных органов которых входят дисквалифицированные лица

https://www.nalog.gov.ru/opendata/7707329152-disqualifiedpersons/

- csv на около 5 тыс строчек: - те в общем масштабе их мало
    - Полное наименование
    - ОГРН
    - ИНН
    - КПП
    - Адрес 

- беру только список инн (здесь только уникальные значения), будет флагом

In [8]:
with_d_person = pd.read_csv('raw_data/with_disqual_person.csv', sep=';', dtype={'G3': str})
with_d_person.head(3)

,G1,G2,G3,G4,G5
0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КАРТЕН""",1175749002263,5751060080,575101001,"302030,ОРЛОВСКАЯ ОБЛАСТЬ,,ОРЁЛ ГОРОД,,МОСКОВСК..."
1,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВЕПРЬ""",1087205000541,7222018450,722201001,"627500,ТЮМЕНСКАЯ ОБЛАСТЬ,СОРОКИНСКИЙ РАЙОН,,БО..."
2,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САМАН""",1190280053732,0272913945,27201001,"450095,БАШКОРТОСТАН РЕСПУБЛИКА,,УФА ГОРОД,,МАЙ..."


In [9]:
inn_disqual = tuple(with_d_person.G3)
len(inn_disqual)

4916

#### - Сведения о ценах и объемах реализации продуктовых товаров и горюче-смазочных материалов в субъектах Российской Федерации по данным контрольно-кассовой техники.

https://www.nalog.gov.ru/opendata/7707329152-fnsprice/


- можно соединять по регионам: стоимость потребительской корзины и стоимость gsm
- за периоды 2019, 2020, 2021 - берём среднее за год

- в описании структуры файла немного перепутаны столбцы (дата и код региона)...

    - GA - код региона (1, 2, 3...) -в кодировке ФНС России
    - G2 - Дата наблюдения
    - G35 - Стоимость потребительской корзины для трудоспособных граждан
    - G36 - Стоимость потребительской корзины для пенсионеров
    - G37 - Стоимость потребительской корзины для детей до 18 лет
    - G70 - Бензин марки АИ-92, руб/литр
    - G71 - Бензин марки АИ-95, руб/литр
    - G72 - Бензин марки АИ-98, руб/литр
    - G73 - Дизельное топливо, руб/литр
    - G74 - Бензин марки АИ-92, (Объем проданной продукции)
    - G75 - Бензин марки АИ-95, (Объем проданной продукции)
    - G76 - Бензин марки АИ-98, (Объем проданной продукции)
    - G77 - Дизельное топливо, (Объем проданной продукции)

In [10]:
data = pd.read_csv('raw_data/prices_by_regions/data-20220331-structure-20220331.csv', sep=';')

In [11]:
# отбираем и переименовываем столбцы
groc_and_fuels = data[['GA', 'G2', 'G35', 'G36', 'G37', 'G70', 'G71', 'G72', 'G73', 'G74', 
                       'G75', 'G76', 'G77']]
groc_and_fuels.columns = ['region_id', 'observ_date', 'cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']

In [12]:
groc_and_fuels.head(3)

,region_id,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
0,1,01.01.2019,"5023,51","4301,36","5001,97","42,96","46,87","50,2","45,53","103778,69","56968,84","121,19","13973,49"
1,1,02.01.2019,"5086,32",4354,"5064,53","42,98","46,92","50,3","45,38","185820,84","119512,26","95,09","39283,9"
2,1,03.01.2019,"4930,56","4214,3","4911,48","41,13","46,86","50,77","45,46","219913,38","126519,19","107,82","45498,67"


In [13]:
# все цены распознаёт как строку
groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100810 entries, 0 to 100809
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   region_id                100810 non-null  int64 
 1   observ_date              100810 non-null  object
 2   cons_basket_working_age  100810 non-null  object
 3   cons_basket_pens         100810 non-null  object
 4   cons_basket_child        100810 non-null  object
 5   AI92_cost                100810 non-null  object
 6   AI95_cost                100810 non-null  object
 7   AI98_cost                100810 non-null  object
 8   diesel_cost              100810 non-null  object
 9   AI92_volume              100810 non-null  object
 10  AI95_volume              100810 non-null  object
 11  AI98_volume              100810 non-null  object
 12  diesel_volume            100810 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.0+ MB


In [14]:
# заменяем запятую на точку и приводим к float
for col in groc_and_fuels.columns[2:]:
    groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)

# добавляем год (чтобы усреднить за период)
groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year
groc_and_fuels.head(3)

C:\Users\013\AppData\Local\Temp/ipykernel_3088/1479746939.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)
C:\Users\013\AppData\Local\Temp/ipykernel_3088/1479746939.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year


,region_id,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume,observ_year
0,1,01.01.2019,5023.51,4301.36,5001.97,42.96,46.87,50.20,45.53,103778.69,56968.84,121.19,13973.49,2019
1,1,02.01.2019,5086.32,4354.00,5064.53,42.98,46.92,50.30,45.38,185820.84,119512.26,95.09,39283.90,2019
2,1,03.01.2019,4930.56,4214.30,4911.48,41.13,46.86,50.77,45.46,219913.38,126519.19,107.82,45498.67,2019


In [15]:
groc_and_fuels.observ_year.unique()

array([2019, 2020, 2021, 2022], dtype=int64)

In [16]:
groc_and_fuels.describe()

,region_id,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume,observ_year
count,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,1.008100e+05,1.008100e+05,100810.000000,1.008100e+05,100810.000000
mean,43.388235,5727.310486,4878.928747,5748.071100,43.652529,47.168287,40.781701,48.460662,5.434237e+05,4.326813e+05,5607.199239,1.631631e+05,2020.151771
std,25.201476,1374.112292,1161.590332,1474.763687,4.806918,4.936959,29.591852,6.595098,6.217567e+05,7.020778e+05,14599.542048,2.647617e+05,0.946589
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,2019.000000
25%,22.000000,4953.200000,4225.492500,4963.042500,41.840000,45.280000,46.060000,46.390000,1.944471e+05,1.287709e+05,36.740000,3.815213e+04,2019.000000
50%,43.000000,5493.475000,4680.695000,5481.580000,43.270000,46.910000,50.380000,48.020000,3.439578e+05,2.297549e+05,928.335000,8.911888e+04,2020.000000
75%,64.000000,6206.660000,5282.115000,6162.002500,45.440000,49.080000,53.970000,50.920000,6.772496e+05,4.597270e+05,4798.805000,1.751593e+05,2021.000000
max,92.000000,20292.650000,16722.400000,19410.610000,60.740000,63.000000,1000.000000,70.470000,5.867661e+06,7.963218e+06,293081.090000,2.965140e+06,2022.000000


- есть наблюдения с нулевыми значениями - это скорее всего должны быть None
- сперва заменяю нули на Nan, а потом агрегирую (mean игнорирует Nan при расчёте среднего)

In [17]:
groc_and_fuels = groc_and_fuels.replace(0, None)

In [18]:
avg_groc_and_fuels = groc_and_fuels.groupby(['region_id', 'observ_year'], as_index=False)[['cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']].mean()
avg_groc_and_fuels.head(3)

,region_id,observ_year,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
0,1,2019,5005.002466,4269.761836,4923.843233,43.332219,48.051562,50.837918,45.404822,221138.361425,122866.607068,159.544000,65073.028384
1,1,2020,5146.416749,4387.573552,5092.815164,43.547923,48.533852,52.076913,46.012650,187478.847978,123215.335929,72.161475,60822.033087
2,1,2021,5914.448685,5031.818411,5815.689205,44.553863,50.663260,54.583315,47.691973,208157.641918,142313.449507,1248.010493,69281.618521


In [19]:
avg_groc_and_fuels.describe()

,region_id,observ_year,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,3.400000e+02,3.400000e+02,340.000000,3.400000e+02
mean,43.388235,2020.500000,6015.032872,5122.777485,6036.548506,44.703428,48.314539,58.033460,50.217531,5.254999e+05,4.289553e+05,7487.232891,1.651926e+05
std,25.238494,1.119682,1303.370487,1100.238105,1419.839363,3.459869,3.419028,27.020610,4.215236,5.988471e+05,6.889907e+05,17352.852778,2.588695e+05
min,1.000000,2019.000000,4248.657425,3623.140493,4258.991534,30.757459,42.680986,40.564110,42.806658,2.524374e+03,1.060030e+03,3.230874,7.202673e+02
25%,22.000000,2019.750000,5063.237383,4320.612277,5086.709206,42.195976,45.738340,50.108363,46.782692,1.874805e+05,1.304321e+05,748.641602,4.623857e+04
50%,43.000000,2020.500000,5804.088227,4943.945219,5736.621575,44.311764,47.980808,53.768740,49.175219,3.360336e+05,2.271063e+05,2460.292753,9.344334e+04
75%,64.000000,2021.250000,6508.271664,5532.384874,6447.250315,46.511551,50.319172,58.316250,52.444556,6.592407e+05,4.450006e+05,6820.945889,1.808378e+05
max,92.000000,2022.000000,14021.350778,11865.518556,15757.352444,60.320110,63.000000,361.476192,65.622111,4.283245e+06,5.545636e+06,195210.374889,1.936039e+06


- посмотрим совпадения кодов регионов в csv и в postgresql (поле, по которому будет соединение)

In [21]:
groc_and_fuels.region_id.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 86, 87, 89, 91, 92],
      dtype=int64)

In [ ]:
# уникальные из postgre
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [ ]:
with connect:
    with connect.cursor() as cur:
        select = 'SELECT DISTINCT region_code FROM reestr_msp ORDER BY region_code;'
        cur.execute(select)
        print(cur.fetchall())

In [ ]:
connect.close()

- код 93 в postgre - Краснодарский край, соответствует 23 в groc_and_fuels),
- 94 в postgre (Территории, находящиеся за пределами РФ и обслуживаемые Управлением режимных объектов МВД России, Байконур), в groc_and_fuels данных нет
- 95 - Чеченская Республика - соответствует код 20 в groc_and_fuels

- интересный код 0: в postgre ему соответствуют 23 inn, ни для одного из которых нет отчётов в открытых данных (на bo.nalog)

#### - Таблица 5. Численность населения России, федеральных округов, субъектов Российской Федерации, городских округов, муниципальных районов, муниципальных округов, городских и сельских поселений, городских населенных пунктов, сельских населенных пунктов с населением 3000 человек и более

https://rosstat.gov.ru/vpn_popul

https://rosstat.gov.ru/storage/mediabank/tab-5_VPN-2020.xlsx

- здесь соединять придётся по наименованию
- в таблице - данные по населённым пунктам с численностью более 3000 чел
- таблица неудобная для извлечения - наименования разноформатны
- как оказалось, данные по адресу что в MongoDb, что в xml - не очень полные и не очень корректные: перепутан уровень (город в поле региона - особенно для больших городов (оказалось, что так и есть, большой город = регион, но это оставляет поле города пустым), село в поле города и т.д.), достаточно много пропусков, есть адреса, которые сложно интерпретировать с точки зрения численности населения (50-й километр какого-нибудь шоссе) + номерные города (Чехов-4), по которым нет данных
    - можно заполнить медианой (или средним) всех имеющихся значений численности менее 3 тыс.

- структура адресов из Mongo

In [22]:
client = MongoClient('localhost', 27017)
mongobase = client.bo_nalog
collection = mongobase['bonalogru']

In [23]:
# таблица с адресами - чтобы посмотреть соотношение
addresses = pd.DataFrame.from_records(collection.find({}, {'region': 1, 'district': 1, 'city': 1,
                                                           'settlement': 1}))

In [24]:
addresses.head(3)

,_id,region,district,city,settlement
0,3447001553,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None
1,3447003470,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None
2,3447003247,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None


In [25]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1918648 entries, 0 to 1918647
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   _id         object
 1   region      object
 2   district    object
 3   city        object
 4   settlement  object
dtypes: object(5)
memory usage: 73.2+ MB


In [28]:
# для тех, у кого нет региона, но есть город - регион можно восстановить
# но регион на самом деле не нужен, так как в postgresql в поле код региона нет пропусков
addresses.loc[addresses.region.isna()].head(3)

,_id,region,district,city,settlement
126136,3907027634,None,None,КАЛИНИНГРАД,None
138500,2312124742,None,None,КРАСНОДАР,None
138712,2312124887,None,None,КРАСНОДАР,None


In [27]:
# district - очень мало значений
# а часть из них - города (как Ногинск или Одинцово ниже)
addresses.district.isna().sum()

1650576

- и в целом есть самые разные комбинации пропусков

In [30]:
addresses.loc[~addresses.settlement.isna() & addresses.city.isna()]

,_id,region,district,city,settlement
9,5032184021,МОСКОВСКАЯ,ОДИНЦОВО,None,ВНИИССОК
12,5032206081,МОСКОВСКАЯ,ОДИНЦОВО,None,ЛЕСНОЙ ГОРОДОК
13,5032197768,МОСКОВСКАЯ,ОДИНЦОВО,None,ШАРАПОВКА
26,5032206476,МОСКОВСКАЯ,ОДИНЦОВО,None,ЧАСЦЫ
27,5032194661,МОСКОВСКАЯ,ОДИНЦОВО,None,ШУЛЬГИНО
...,...,...,...,...,...
1918287,1435172399,САХА /ЯКУТИЯ/,УСТЬ-ЯНСКИЙ,None,КАЗАЧЬЕ
1918321,5040095713,КАЛУЖСКАЯ,ХВАСТОВИЧСКИЙ,None,ЛОВАТЬ
1918439,5040095456,МОСКОВСКАЯ,Ленинский,None,Горки
1918477,5040094830,МОСКОВСКАЯ,НАРО-ФОМИНСК,None,СОФЬИНО


In [31]:
# нет ни city ни district
addresses.loc[addresses.settlement.isna() & addresses.city.isna()]

,_id,region,district,city,settlement
7,5032200629,МОСКОВСКАЯ,ОДИНЦОВО,None,None
10,5032190642,МОСКОВСКАЯ,ОДИНЦОВО,None,None
14,5032185280,МОСКОВСКАЯ,ОДИНЦОВО,None,None
15,5032194816,МОСКВА,None,None,None
16,5032205666,МОСКОВСКАЯ,ОДИНЦОВО,None,None
...,...,...,...,...,...
1918598,5040095110,МОСКВА,None,None,None
1918613,5040091652,МОСКВА,None,None,None
1918625,5040101597,МОСКВА,None,None,None
1918641,5040088628,САНКТ-ПЕТЕРБУРГ,None,None,None


In [32]:
# но большую часть можно заполнить - тк город указан в region
addresses.loc[addresses.settlement.isna() & addresses.city.isna()].region.unique()

array(['МОСКОВСКАЯ', 'МОСКВА', 'САНКТ-ПЕТЕРБУРГ', 'ЛЕНИНГРАДСКАЯ',
       'ХАБАРОВСКИЙ', 'СЕВАСТОПОЛЬ', 'САМАРСКАЯ', 'ОРЕНБУРГСКАЯ',
       'КОСТРОМСКАЯ', 'ИРКУТСКАЯ', 'ТАТАРСТАН', 'Севастополь',
       'УЛЬЯНОВСКАЯ', 'ПЕНЗЕНСКАЯ', 'НОВГОРОДСКАЯ', 'РОСТОВСКАЯ',
       'ПЕРМСКИЙ', 'УДМУРТСКАЯ', 'СВЕРДЛОВСКАЯ', 'ЛИПЕЦКАЯ', 'ИВАНОВСКАЯ',
       'БАШКОРТОСТАН', 'НОВОСИБИРСКАЯ', 'ПРИМОРСКИЙ', 'САРАТОВСКАЯ',
       'ТЮМЕНСКАЯ', 'ТАМБОВСКАЯ', 'ТВЕРСКАЯ', 'КАРЕЛИЯ', 'ЗАБАЙКАЛЬСКИЙ',
       'МУРМАНСКАЯ', 'ВОРОНЕЖСКАЯ', 'КРАСНОДАРСКИЙ', 'КРАСНОЯРСКИЙ',
       'БЕЛГОРОДСКАЯ', 'СТАВРОПОЛЬСКИЙ', 'НИЖЕГОРОДСКАЯ', 'КУРГАНСКАЯ',
       'ТОМСКАЯ', 'ТУЛЬСКАЯ', 'МАРИЙ ЭЛ', 'АСТРАХАНСКАЯ', 'КАЛУЖСКАЯ',
       'СМОЛЕНСКАЯ', 'ВЛАДИМИРСКАЯ', 'КАМЧАТСКИЙ', 'КОМИ', 'ЧЕЛЯБИНСКАЯ',
       'КАЛИНИНГРАДСКАЯ', 'БУРЯТИЯ', 'Санкт-Петербург', 'АЛТАЙ', 'Москва',
       'КРЫМ', 'КИРОВСКАЯ', 'САХАЛИНСКАЯ', 'РЯЗАНСКАЯ', 'Московская',
       'КУРСКАЯ', 'ПСКОВСКАЯ', 'АРХАНГЕЛЬСКАЯ',
       'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ

In [33]:
# Москва
addresses.loc[addresses.settlement.isna() & addresses.city.isna()].loc[addresses.region=='МОСКВА'].shape

(322270, 5)

- для проверки - в xml действительно в поле Регион указан Город

      <Регион Наим="САНКТ-ПЕТЕРБУРГ" Тип="ГОРОД"/> 

- т.е. придётся заполнять пропуски и выбирать способ, как определить конечный пункт, по которому соединять с таблицей с численностью
- пока - уникальные city и settlement из Mongo

In [34]:
cities = collection.distinct('city')
cities = set(el.title() for el in cities[1:])

settlements = collection.distinct('settlement')
settlements = set(el.title() for el in settlements[1:])

len(cities), len(settlements)

(1298, 23372)

#### данные переписи

In [35]:
data = pd.read_excel('raw_data/tab-5_VPN-2020.xlsx', header=3)
population = data[['Unnamed: 0', 'Unnamed: 1']].rename(columns={'Unnamed: 0': 'locality', 'Unnamed: 1': 'population'})

In [36]:
# строки разного уровня + очень неединообразные наименования
# для города: город, г. г и т.д.
population

,locality,population
0,Российская Федерация,147182123
1,Городское население,110075322
2,Сельское население,37106801
3,Центральный федеральный округ,40334532
4,Городское население,33118003
...,...,...
27794,Сельское поселение Лаврентия,1420
27795,Сельское поселение Лорино,1426
27796,Сельское поселение Нешкан,712
27797,Сельское поселение Уэлен,685


In [37]:
# порядок важен - в городах сперва ищем город!
def get_population(name):
    val = population.loc[population['locality']\
          .str \
          .contains(rf'г. {name}\b|пгт {name}\b|село {name}\b|пос[ёе]лок {name}\b|п. {name}\b|[Сс]ельское поселение {name}\b|^{name}\b сельское поселение|Поселение {name}\b|[Дд]еревня {name}\b|г {name}\b|город {name}\b')]\
          .population.values
    return val[0] if val.any() else None

In [38]:
%%time
city_to_population = {key: get_population(key) for key in cities}

Wall time: 1min 47s


In [40]:
# часть на найденных можно всё-таки найти, если очистить данные
# словарь для замены в датасете: - надо будет обновить, так как данные ещё добавляются
city_replace = {
    'Королев': 'Королёв',
    'Белохолуницкое Городское Поселение': 'Белохолуницкое',
    'Мо Сельское Поселение Следневское': 'Следневское',
    'Камень-На-Оби': 'Камень-на-Оби',
    'Широкобуеракское Мо': 'Широкобуеракское',
    'Новопушкинское Мо': 'Новопушкинское',
    'Ростов-На-Дону': 'Ростов-на-Дону',
    'Сольцы 2': 'Сольцы',
    'Буденновск': 'Будённовск',
    'Городское Поселение Снегири': 'Снегири',
    'Калач-На-Дону': 'Калач-на-Дону',
    'Славянск-На-Кубани': 'Славянск-на-Кубани',
    'Терсинское Мо': 'Терсинское',
    'Николаевск-На-Амуре': 'Николаевск-на-Амуре',
    'Комсомольск-На-Амуре': 'Комсомольск-на-Амуре',
}

- для settlement - можно искать таким же образом (только совпадений намного меньше и поиск заметно дольше) - из 11 тыс уникальных всего около 2 тыс значений численности
- большая часть из этих None - менее 3 тыс населения!

#### В итоге:

- довольно большая часть данных будет коррелирована (так как есть обобщённые столбцы и столбцы, конкретизирующие обобщённую информацию) - те придётся ещё отбирать

- данные обновляются достаточно редко (для большей части орагнизаций - только ежегодные публичные отчёты) + время выкладки

- я не учитываю коррекции и поправки данных - а для них предусмотрены свои формы

#### 1.4. Соединение таблиц в postgre - необязательная часть

- чтуобы установить внешний ключ в postgre по inn, в дочерних таблицах не должно быть inn, которых нет в родительской (reestr_msp)
- это долгий запрос в Postgre, поэтому пробую найти нужные inn в python и потом удалить их

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [ ]:
connect.close()

In [ ]:
%%time
with connect:
    with connect.cursor() as cur:
        cur.execute('SELECT inn FROM reestr_msp')
        reestr_inn = cur.fetchall()
        cur.execute('SELECT inn FROM debt')
        debt_inn = cur.fetchall()
        cur.execute('SELECT inn FROM tax_offense')
        tax_offense_inn = cur.fetchall()
        # здесь inn не уникален!
        cur.execute('SELECT DISTINCT inn FROM support_msp')
        support_msp_inn = cur.fetchall()

In [ ]:
reestr_inn = np.array(reestr_inn)[:, 0]
debt_inn = np.array(debt_inn)[:, 0]
tax_offense_inn = np.array(tax_offense_inn)[:, 0]
support_msp_inn = np.array(support_msp_inn)[:, 0]

In [ ]:
len(reestr_inn), len(debt_inn), len(tax_offense_inn), len(support_msp_inn)

In [ ]:
# находим разницу # это те, кто не является малым и средним, или кого уже удалили из реестра...
debt_diff = tuple(set(debt_inn) - set(reestr_inn))
tax_offense_diff = tuple(set(tax_offense_inn) - set(reestr_inn))
support_diff = tuple(set(support_msp_inn) - set(reestr_inn))

In [ ]:
len(debt_diff), len(tax_offense_diff), len(support_diff)

- удаление

In [ ]:
%%time
with connect:
    with connect.cursor() as cur:
        cur.execute(sql.SQL('DELETE FROM {} WHERE inn IN {}') \
                          .format(sql.Identifier('debt'), 
                                  sql.Literal(debt_diff)))
        cur.execute(sql.SQL('DELETE FROM {} WHERE inn IN {}') \
                          .format(sql.Identifier('tax_offense'), 
                                  sql.Literal(tax_offense_diff)))
        cur.execute(sql.SQL('DELETE FROM {} WHERE inn IN {}') \
                          .format(sql.Identifier('support_msp'), 
                                  sql.Literal(support_diff)))

- теперь можно устанавливать ограничение

In [ ]:
with connect:
    with connect.cursor() as cur:
        cur.execute('ALTER TABLE debt ADD FOREIGN KEY (inn) REFERENCES reestr_msp (inn)')
        cur.execute('ALTER TABLE tax_offense ADD FOREIGN KEY (inn) REFERENCES reestr_msp (inn)')
        cur.execute('ALTER TABLE support_msp ADD FOREIGN KEY (inn) REFERENCES reestr_msp (inn)')    

### 2. Соединение датасета

- на этапе соединения - 1 337 681 сырых строк из MongoDB, данные ещё собираются

In [2]:
with open('parsed_data/mongo_data.pickle', 'rb') as f:
    mongo_data = pickle.load(f)

In [3]:
mongo_data

,_id,id_org,shortName,ogrn,index,region,district,city,settlement,okved,...,2020_2350,2021_2300,2020_2300,2021_2410,2020_2410,2021_2400,2020_2400,2021_2500,2020_2500,2021_1220
0,3447001553,5581428,"ООО ""МОДЕРН-БОН""",1023404293505,400066,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,0.0,NaN,NaN,138.0,0.0,1270.0,0.0,NaN,NaN,NaN
1,3447003470,5573882,"ООО ""МАРКО""",1023404287829,400079,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,NaN,-29.0,30.0,NaN,NaN,-29.0,30.0,-29.0,30.0,NaN
2,3447003247,6838461,"ООО ""ОКСИ""",1023444287855,400057,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,83.0,NaN,NaN,152.0,113.0,1099.0,691.0,NaN,NaN,NaN
3,3447003416,6838444,"ОАО ""ФИРМА ЖБИ-6""",1023444287580,400057,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,1586.0,24837.0,11070.0,-5181.0,-2283.0,19656.0,8787.0,17783.0,7599.0,NaN
4,3447002444,5581537,"ПК ""МОНОЛИТ""",1023404293725,400059,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,NaN,0.0,4.0,NaN,NaN,0.0,4.0,0.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918643,5040094371,9220029,"ООО ""ПРОФСТРОЙ""",1155040003150,140182,МОСКОВСКАЯ,None,ЖУКОВСКИЙ,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1918644,5040092303,7220381,"ООО ""КЛИНИК""",1095040003728,140188,МОСКОВСКАЯ,None,ЖУКОВСКИЙ,None,None,...,163.0,NaN,NaN,589.0,410.0,5438.0,1982.0,NaN,NaN,NaN
1918645,5040096932,7271929,"ООО ""ЛАРИКС ХАУС""",1105040000658,140145,МОСКОВСКАЯ,None,РАМЕНСКОЕ,ГЖЕЛЬ,None,...,1016.0,NaN,NaN,259.0,75.0,-13381.0,-6178.0,NaN,NaN,NaN
1918646,5040094861,4132796,"ООО ЧОО ""О.С.А.-ГАРАНТ""",1095040006160,140145,МОСКОВСКАЯ,None,РАМЕНСКОЕ,ТРОШКОВО,None,...,828.0,NaN,NaN,168.0,180.0,2593.0,2451.0,NaN,NaN,NaN


- оставляю нужные столбцы, привожу типы - чтобы уменьшить размер
- shortname и _id в датасет не попадут, они просто для удобства

In [4]:
columns = ['_id', 'shortName', 'index', 'region', 'city', 'settlement', 'okved2', 'okopf',
           'statusCode', 'registrationDate', 'authorizedCapital', '2021_1150', '2020_1150',
           '2019_1150', '2021_1170', '2020_1170', '2019_1170', '2021_1210', '2020_1210',
           '2019_1210', '2021_1230', '2020_1230', '2019_1230', '2021_1250', '2020_1250',
           '2019_1250', '2021_1600', '2020_1600', '2019_1600', '2021_1350', '2020_1350',
           '2019_1350', '2021_1360', '2020_1360', '2019_1360', '2021_1300', '2020_1300',
           '2019_1300', '2021_1520', '2020_1520', '2019_1520', '2021_1700', '2020_1700',
           '2019_1700', '2021_2110', '2020_2110', '2021_2120', '2020_2120', '2021_2330',
           '2020_2330', '2021_2340', '2020_2340', '2021_2350', '2020_2350', '2021_2410',
           '2020_2410', '2021_2400', '2020_2400',
          ]
mongo_data = mongo_data[columns]

In [5]:
mongo_data.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1918648 entries, 0 to 1918647
Data columns (total 58 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   _id                1918648 non-null  object 
 1   shortName          1918648 non-null  object 
 2   index              1912487 non-null  object 
 3   region             1918593 non-null  object 
 4   city               1273534 non-null  object 
 5   settlement         197676 non-null   object 
 6   okved2             1917116 non-null  object 
 7   okopf              1893702 non-null  float64
 8   statusCode         1918648 non-null  object 
 9   registrationDate   1918647 non-null  object 
 10  authorizedCapital  1838219 non-null  float64
 11  2021_1150          828464 non-null   float64
 12  2020_1150          680998 non-null   float64
 13  2019_1150          630704 non-null   float64
 14  2021_1170          221453 non-null   float64
 15  2020_1170          154363 non-nu

- на всякий случай чистим инн

In [6]:
mongo_data._id = mongo_data._id.apply(lambda x: x.strip())
mongo_data._id.nunique(), mongo_data._id.size

C:\Users\013\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


(1918648, 1918648)

- окопф - заполняем пропуски 0 и приводим к object
- в дальнейшем оказалось, что удобнее (быстрее) работать как с цифрой, а потеря первого нуля не критична

In [7]:
mongo_data.okopf = mongo_data.okopf.fillna(0).astype('int32').astype('object')

- дату - к datetime

In [8]:
mongo_data.registrationDate = pd.to_datetime(mongo_data.registrationDate)

- посмотрим, удастся ли привести какой-нибудь float64 к float32

In [9]:
for col in mongo_data.columns[10:]:
    mongo_data[col] = pd.to_numeric(mongo_data[col], downcast='float')

C:\Users\013\AppData\Local\Temp/ipykernel_2988/315581999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mongo_data[col] = pd.to_numeric(mongo_data[col], downcast='float')


In [10]:
mongo_data.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1918648 entries, 0 to 1918647
Data columns (total 58 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   _id                1918648 non-null  object        
 1   shortName          1918648 non-null  object        
 2   index              1912487 non-null  object        
 3   region             1918593 non-null  object        
 4   city               1273534 non-null  object        
 5   settlement         197676 non-null   object        
 6   okved2             1917116 non-null  object        
 7   okopf              1918648 non-null  object        
 8   statusCode         1918648 non-null  object        
 9   registrationDate   1918647 non-null  datetime64[ns]
 10  authorizedCapital  1838219 non-null  float32       
 11  2021_1150          828464 non-null   float32       
 12  2020_1150          680998 non-null   float32       
 13  2019_1150          630704 n

#### - дальше забираем вторую большую таблицу - реестр мсп из PostgreSQL. 

- нужна не вся таблица, а только те инн, которые есть в Mongo - но, кажется, отбросить лишние строки будет быстрее в python.

In [11]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [87]:
connect.close()

In [12]:
columns = [
    'inn', 
    'n_employees', 
    'is_soc_enterprise', 
    'newly_establ', 
    'subj_cat', 
    'subj_form', 
    'inclusion_date',  # отличается от registration_date! - сравнить!
    'region_code', 
    'n_okovd_add',  
#     'okovd_add_vers',  # только убедиться, что везде одно значение! - везде, где есть, одно
#     'okovd_add_code',  # можно придумать много интересных признаков
    'n_licenses',
    'n_products',
#     'products_code'  # то же самое
    'is_high_tech',
    'n_contracts',
    'n_agreements'
]

In [13]:
%%time
reestr_msp = pd.read_sql(sql.SQL('SELECT {} FROM reestr_msp;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

Wall time: 16.3 s


In [14]:
reestr_msp.head()

,inn,n_employees,is_soc_enterprise,newly_establ,subj_cat,subj_form,inclusion_date,region_code,n_okovd_add,n_licenses,n_products,is_high_tech,n_contracts,n_agreements
0,3447001553,1.0,0,0,1,1,2016-08-01,34,29,0,0,0,0,0
1,3447003470,2.0,0,0,1,1,2016-08-01,34,6,0,0,0,0,0
2,3447003247,18.0,0,0,1,1,2016-08-01,34,24,1,0,0,0,0
3,3447003416,165.0,0,0,3,1,2018-08-10,34,9,2,0,0,0,0
4,3447002444,NaN,0,0,1,1,2016-08-01,34,0,0,0,0,0,0


- здесь инн чистые - это уникальный ключ
- оставляем только нужные строки

In [15]:
reestr_msp.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332299 entries, 0 to 2332298
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   inn                2332299 non-null  object 
 1   n_employees        1938849 non-null  float64
 2   is_soc_enterprise  2332299 non-null  int64  
 3   newly_establ       2332299 non-null  int64  
 4   subj_cat           2332299 non-null  int64  
 5   subj_form          2332299 non-null  int64  
 6   inclusion_date     2332299 non-null  object 
 7   region_code        2332299 non-null  int64  
 8   n_okovd_add        2332299 non-null  int64  
 9   n_licenses         2332299 non-null  int64  
 10  n_products         2332299 non-null  int64  
 11  is_high_tech       2332299 non-null  int64  
 12  n_contracts        2332299 non-null  int64  
 13  n_agreements       2332299 non-null  int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 249.1+ MB


In [16]:
# они уникальны
actual_inn = tuple(mongo_data._id)

- а это очень интересно, как так вышло, что в реестре меньше инн, чем в данных из интернета: должно быть строго наоборот (так как паук идёт по списку из реестра), если инн нет в реестре, то как его можно было найти на сайте bfo?
- но при обработке xml некоторые данные отбрасывались как некорректные (нет отчётов или важных полей)

In [17]:
len(actual_inn), reestr_msp.loc[reestr_msp.inn.isin(actual_inn)].shape

(1918648, (1918094, 14))

In [18]:
reestr_msp = reestr_msp.loc[reestr_msp.inn.isin(actual_inn)]

reestr_msp.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1918094 entries, 0 to 2332298
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   inn                1918094 non-null  object 
 1   n_employees        1842338 non-null  float64
 2   is_soc_enterprise  1918094 non-null  int64  
 3   newly_establ       1918094 non-null  int64  
 4   subj_cat           1918094 non-null  int64  
 5   subj_form          1918094 non-null  int64  
 6   inclusion_date     1918094 non-null  object 
 7   region_code        1918094 non-null  int64  
 8   n_okovd_add        1918094 non-null  int64  
 9   n_licenses         1918094 non-null  int64  
 10  n_products         1918094 non-null  int64  
 11  is_high_tech       1918094 non-null  int64  
 12  n_contracts        1918094 non-null  int64  
 13  n_agreements       1918094 non-null  int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 219.5+ MB


- в n_employees заполняю Nan как -1 и заодно изменю тип

In [19]:
reestr_msp.n_employees.min(), reestr_msp.n_employees.max()

(0.0, 8617.0)

In [20]:
reestr_msp.n_employees = pd.to_numeric(reestr_msp.n_employees.fillna(-1), downcast='integer')

In [21]:
n_columns = ['is_soc_enterprise', 'newly_establ', 'subj_cat', 'subj_form', 'region_code', 
             'n_products', 'is_high_tech', 'n_okovd_add', 'n_licenses', 'n_contracts', 
             'n_agreements']

In [22]:
for col in n_columns:
    reestr_msp[col] = pd.to_numeric(reestr_msp[col], downcast='integer')

In [23]:
reestr_msp.inclusion_date = pd.to_datetime(reestr_msp.inclusion_date)

In [24]:
reestr_msp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1918094 entries, 0 to 2332298
Data columns (total 14 columns):
 #   Column             Dtype         
---  ------             -----         
 0   inn                object        
 1   n_employees        int16         
 2   is_soc_enterprise  int8          
 3   newly_establ       int8          
 4   subj_cat           int8          
 5   subj_form          int8          
 6   inclusion_date     datetime64[ns]
 7   region_code        int8          
 8   n_okovd_add        int16         
 9   n_licenses         int16         
 10  n_products         int8          
 11  is_high_tech       int8          
 12  n_contracts        int16         
 13  n_agreements       int16         
dtypes: datetime64[ns](1), int16(5), int8(7), object(1)
memory usage: 75.0+ MB


#### - теперь объединим две основные таблицы 


- не знаю, оставлять ли строки, для которых нет соответствия в reestr_msp? 
- пока оставлю и посмотрю на статус этих организаций - так как у меня и так дисбаланс

In [25]:
mongo_data.rename(columns={'_id': 'inn'}, inplace=True)

C:\Users\013\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


- в итоге - не беру пропуски (inner а не left)

In [26]:
%%time
data = mongo_data.merge(reestr_msp, how='inner', on='inn')

Wall time: 4.45 s


- здесь много INACTIVE
- но 400+ пропусков, для которых нет данных из реестра из паука

        ACTIVE               372
        INACTIVE              58
        LIQUIDATION_STAGE      3
        Name: statusCode, dtype: int64

In [28]:
# data.loc[data.n_employees.isna(), 'statusCode'].value_counts()

- но соотношение не сильно меняется...

#### - идём дальше, support_msp 
- здесь надо аггрегировать

In [29]:
columns = [
    'inn', 
    'support_date', 
    'is_breach',  # их и так немного - 890, а при разбивке на периоды - и того меньше...
    'form_code', 
    'support_volume', 
    'support_unit', 
]

In [30]:
%%time
support_msp = pd.read_sql(sql.SQL('SELECT {} FROM support_msp;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

Wall time: 4.19 s


In [31]:
support_msp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109362 entries, 0 to 1109361
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   inn             1109362 non-null  object 
 1   support_date    1109362 non-null  object 
 2   is_breach       1109362 non-null  int64  
 3   form_code       1109362 non-null  object 
 4   support_volume  1109362 non-null  float64
 5   support_unit    1109362 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 50.8+ MB


- сразу удаляем лишние строки

In [32]:
support_msp = support_msp.loc[support_msp.inn.isin(actual_inn)]

support_msp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1081936 entries, 3 to 1109361
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   inn             1081936 non-null  object 
 1   support_date    1081936 non-null  object 
 2   is_breach       1081936 non-null  int64  
 3   form_code       1081936 non-null  object 
 4   support_volume  1081936 non-null  float64
 5   support_unit    1081936 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 57.8+ MB


- добавляем год - чтобы взять только нужный период

In [33]:
support_msp['support_year'] = pd.to_datetime(support_msp.support_date).dt.year

In [34]:
support_msp.support_year.unique()

array([2020, 2021, 2022, 2019, 2023], dtype=int64)

In [35]:
support_msp = support_msp.loc[support_msp.support_year.isin((2021, 2020, 2019))]

- аггрегируем: берём только support_unit = 1

In [36]:
agg_support = pd.pivot_table(support_msp.loc[support_msp.support_unit == 1], values='support_volume', 
               index='inn', columns=['form_code', 'support_year'], aggfunc=sum).reset_index()

In [37]:
agg_support.head(3)

form_code            inn 0101           0102           0103               \
support_year             2019 2020 2021 2019 2020 2021 2019         2020   
0             0101000021  NaN  NaN  NaN  NaN  NaN  NaN  NaN    145560.00   
1             0101000832  NaN  NaN  NaN  NaN  NaN  NaN  NaN    509460.00   
2             0101003086  NaN  NaN  NaN  NaN  NaN  NaN  NaN  15177215.01   

form_code              0105            
support_year      2021 2019 2020 2021  
0                  NaN  NaN  NaN  NaN  
1             243048.0  NaN  NaN  NaN  
2                  NaN  NaN  NaN  NaN

- избавимся от мультииндекса в столбцах

In [38]:
agg_support.columns = ['inn'] + [f'{el[0]}_{str(el[1])}' for el in agg_support.columns.values[1:]]

In [39]:
agg_support.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222802 entries, 0 to 222801
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        222802 non-null  object 
 1   0101_2019  29 non-null      float64
 2   0101_2020  41 non-null      float64
 3   0101_2021  17 non-null      float64
 4   0102_2019  4950 non-null    float64
 5   0102_2020  5799 non-null    float64
 6   0102_2021  9400 non-null    float64
 7   0103_2019  15916 non-null   float64
 8   0103_2020  173835 non-null  float64
 9   0103_2021  118565 non-null  float64
 10  0105_2019  157 non-null     float64
 11  0105_2020  251 non-null     float64
 12  0105_2021  206 non-null     float64
dtypes: float64(12), object(1)
memory usage: 22.1+ MB


- некоторую поддержку сразу нет смысла брать

        0101 - Инвестиции в капитал, убираем, было бы полезно для узкой выборки!
        0102 - гарантии и поручительство
        0103 - субсидии и гранты
        0105 - финансовая аренда - сомнительно, но пока оставим

- на самом деле уже здесь есть вопросы для анализа:

        - кому и сколько инвестируют в капитал?
        - код 0104 - с процентами - финансовая аренда: кому и сколько?

In [40]:
agg_col = ['0102_2019', '0102_2020', '0102_2021', '0103_2019', '0103_2020', 
           '0103_2021', '0105_2019', '0105_2020', '0105_2021']
agg_support = agg_support[['inn'] + agg_col]

- смотрим, можно ли привести типы:

In [41]:
for col in agg_support.columns[1:]:
    agg_support[col] = pd.to_numeric(agg_support[col], downcast='float')

In [42]:
agg_support.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222802 entries, 0 to 222801
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        222802 non-null  object 
 1   0102_2019  4950 non-null    float32
 2   0102_2020  5799 non-null    float32
 3   0102_2021  9400 non-null    float32
 4   0103_2019  15916 non-null   float32
 5   0103_2020  173835 non-null  float32
 6   0103_2021  118565 non-null  float32
 7   0105_2019  157 non-null     float32
 8   0105_2020  251 non-null     float32
 9   0105_2021  206 non-null     float32
dtypes: float32(9), object(1)
memory usage: 9.3+ MB


- есть странные min значения поддержки, средннее и медиана вроде адекватны

In [43]:
agg_support.describe()

,0102_2019,0102_2020,0102_2021,0103_2019,0103_2020,0103_2021,0105_2019,0105_2020,0105_2021
count,4.950000e+03,5.799000e+03,9.400000e+03,1.591600e+04,1.738350e+05,1.185650e+05,1.570000e+02,2.510000e+02,2.060000e+02
mean,2.555942e+07,2.359572e+07,2.039302e+07,4.795107e+06,7.933884e+05,8.266777e+05,2.105417e+07,1.229704e+07,1.320892e+07
std,7.008282e+07,6.072343e+07,6.319130e+07,1.638677e+07,8.327758e+06,7.427538e+06,3.870494e+07,2.261722e+07,2.057941e+07
min,4.575000e+03,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e-02,1.000000e-01,4.878713e+04,1.116072e+04,2.322478e+04
25%,3.000000e+06,3.000000e+06,2.753750e+06,2.700865e+05,4.852000e+04,2.558400e+04,2.487634e+06,2.556284e+06,3.638371e+06
50%,7.848092e+06,8.400000e+06,7.450000e+06,1.015756e+06,1.213000e+05,7.675200e+04,7.443578e+06,5.000000e+06,6.615283e+06
75%,2.100000e+07,2.100000e+07,1.930950e+07,3.413380e+06,3.396400e+05,2.491660e+05,1.666988e+07,1.021730e+07,1.362242e+07
max,1.912327e+09,1.844604e+09,2.674489e+09,7.482250e+08,1.794792e+09,1.833251e+09,2.292614e+08,1.984200e+08,1.641248e+08


In [44]:
agg_support.shape

(222802, 10)

In [45]:
agg_support.inn.nunique()

222802

- отдельно аггрегируем - были ли нарушения условий поддержки (просто сумма по годам)
- хотя можно было одной таблицей (values списком, aggfunc словарём), но если что - проще убрать

In [46]:
is_breach_agg = support_msp.groupby(['inn', 'support_year'])['is_breach'].sum().reset_index()

In [47]:
is_breach_agg.head(3)

,inn,support_year,is_breach
0,0101000021,2020,0
1,0101000021,2021,0
2,0101000078,2020,0


In [48]:
pivot_is_breach =  pd.pivot_table(is_breach_agg, values='is_breach', 
               index='inn', columns=['support_year'], aggfunc=sum, fill_value=0).reset_index()

In [49]:
pivot_is_breach.head()

support_year,inn,2019,2020,2021
0,0101000021,0,0,0
1,0101000078,0,0,0
2,0101000832,0,0,0
3,0101002861,0,0,0
4,0101003086,0,0,0


In [50]:
pivot_is_breach.loc[pivot_is_breach[2020] > 0].shape

(256, 4)

- переименовываем столбцы и приводим типы

In [51]:
pivot_is_breach.rename(columns={2019: 'breach_2019', 2020: 'breach_2020', 2021: 'breach_2021'}, 
                      inplace=True)

In [52]:
pivot_is_breach.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315944 entries, 0 to 315943
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   inn          315944 non-null  object
 1   breach_2019  315944 non-null  int64 
 2   breach_2020  315944 non-null  int64 
 3   breach_2021  315944 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 9.6+ MB


#### - соединяем и заполняем пропуски нулями

In [53]:
data.shape

(1918094, 71)

In [54]:
%%time
data = data.merge(agg_support, how='left', on='inn')
data = data.merge(pivot_is_breach, how='left', on='inn')

Wall time: 6.21 s


In [55]:
data.shape

(1918094, 83)

In [56]:
for col in agg_col + ['breach_2019', 'breach_2020', 'breach_2021']:
    data[col].fillna(0, inplace=True)

In [57]:
for col in ['breach_2019', 'breach_2020', 'breach_2021']:
    data[col] = pd.to_numeric(data[col], downcast='integer')

#### - следующая таблица - debt - налоговая задолженность

In [58]:
columns = [
    'inn', 
#     'status_date',  # [datetime.date(2021, 12, 31)]
    'arrears', 
    'penalties', 
    'fines', 
]

In [59]:
debt = pd.read_sql(sql.SQL('SELECT {} FROM debt;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

In [60]:
debt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816569 entries, 0 to 816568
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        816569 non-null  object 
 1   arrears    816569 non-null  float64
 2   penalties  816569 non-null  float64
 3   fines      816569 non-null  float64
dtypes: float64(3), object(1)
memory usage: 24.9+ MB


- просто приведём типы, данные уже аггрегированы

In [61]:
for col in debt.columns[1:]:
    debt[col] = pd.to_numeric(debt[col], downcast='float')

In [62]:
debt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816569 entries, 0 to 816568
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        816569 non-null  object 
 1   arrears    816569 non-null  float32
 2   penalties  816569 non-null  float32
 3   fines      816569 non-null  float32
dtypes: float32(3), object(1)
memory usage: 15.6+ MB


#### - соединяем и заполняем пропуски 0

In [63]:
data.shape

(1918094, 83)

In [64]:
%%time
data = data.merge(debt, how='left', on='inn')

Wall time: 3.64 s


In [65]:
data.shape

(1918094, 86)

In [66]:
for col in debt.columns[1:]:
    data[col].fillna(0, inplace=True)

#### - таблица tax_offense - налоговые нарушения

In [67]:
columns = [
    'inn', 
#     'offense_date',  # одна для всех - [datetime.date(2021, 12, 31)]
    'fine_amount', 
]

In [68]:
tax_offense = pd.read_sql(sql.SQL('SELECT {} FROM tax_offense;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

In [69]:
tax_offense.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62723 entries, 0 to 62722
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   inn          62723 non-null  object 
 1   fine_amount  62723 non-null  float64
dtypes: float64(1), object(1)
memory usage: 980.2+ KB


In [70]:
tax_offense.fine_amount = pd.to_numeric(tax_offense.fine_amount, downcast='float')

#### - соединяем

In [71]:
%%time
data = data.merge(tax_offense, how='left', on='inn')

Wall time: 3.1 s


In [72]:
data.fine_amount.fillna(0, inplace=True)

#### -  inn_disqual - просто флаг
- Юридические лица, в состав исполнительных органов которых входят дисквалифицированные лица

        G3 здесь - инн

In [73]:
with_d_person = pd.read_csv('raw_data/with_disqual_person.csv', sep=';', dtype={'G3': str})
with_d_person.head(3)

,G1,G2,G3,G4,G5
0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КАРТЕН""",1175749002263,5751060080,575101001,"302030,ОРЛОВСКАЯ ОБЛАСТЬ,,ОРЁЛ ГОРОД,,МОСКОВСК..."
1,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВЕПРЬ""",1087205000541,7222018450,722201001,"627500,ТЮМЕНСКАЯ ОБЛАСТЬ,СОРОКИНСКИЙ РАЙОН,,БО..."
2,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САМАН""",1190280053732,0272913945,27201001,"450095,БАШКОРТОСТАН РЕСПУБЛИКА,,УФА ГОРОД,,МАЙ..."


In [74]:
# все инн уникальные
with_d_person.G3.nunique()

4916

In [75]:
inn_disqual = tuple(with_d_person.G3)

In [82]:
data['disqual_person'] = 1

In [83]:
%%time
data.disqual_person.where(data.inn.isin(inn_disqual), 0, inplace=True)

Wall time: 189 ms


- слишком большой дисбаланс?
- но пусть пока побудет

In [84]:
data.disqual_person.value_counts()

0    1917853
1        241
Name: disqual_person, dtype: int64

In [86]:
data.disqual_person = pd.to_numeric(data.disqual_person, downcast='integer')

- временно сохраняю в файл

In [ ]:
# with open('parsed_data/data.pickle', 'wb') as f:
#     pickle.dump(data, f)

In [ ]:
with open('parsed_data/data.pickle', 'rb') as f:
    data = pickle.load(f)

#### - добавляем groc_and_fuels

In [90]:
# пропуски и уникальные коды регионов из data
data.region_code.isna().sum(), np.sort(data.region_code.unique())

(0,
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
        69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 86, 87, 89, 90, 91,
        92, 95], dtype=int8))

- 93 соотв 92, 95 соотв 20, 94 - нет данных
- здесь только код 95, переименовывать в столбце не хочу - просто добавлю значение отдельно

In [91]:
data.loc[data.region_code == '94']

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,0105_2020,0105_2021,breach_2019,breach_2020,breach_2021,arrears,penalties,fines,fine_amount,disqual_person


In [92]:
prices_by_regions = pd.read_csv('raw_data/prices_by_regions/data-20220331-structure-20220331.csv', sep=';')

In [93]:
# отбираем и переименовываем столбцы
groc_and_fuels = prices_by_regions[['GA', 'G2', 'G35', 'G36', 'G37', 'G70', 'G71', 'G72', 'G73', 
                                    'G74', 'G75', 'G76', 'G77']]
groc_and_fuels.columns = ['region_code', 'observ_date', 'cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']

In [94]:
groc_and_fuels.head(3)

,region_code,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
0,1,01.01.2019,"5023,51","4301,36","5001,97","42,96","46,87","50,2","45,53","103778,69","56968,84","121,19","13973,49"
1,1,02.01.2019,"5086,32",4354,"5064,53","42,98","46,92","50,3","45,38","185820,84","119512,26","95,09","39283,9"
2,1,03.01.2019,"4930,56","4214,3","4911,48","41,13","46,86","50,77","45,46","219913,38","126519,19","107,82","45498,67"


In [95]:
groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100810 entries, 0 to 100809
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   region_code              100810 non-null  int64 
 1   observ_date              100810 non-null  object
 2   cons_basket_working_age  100810 non-null  object
 3   cons_basket_pens         100810 non-null  object
 4   cons_basket_child        100810 non-null  object
 5   AI92_cost                100810 non-null  object
 6   AI95_cost                100810 non-null  object
 7   AI98_cost                100810 non-null  object
 8   diesel_cost              100810 non-null  object
 9   AI92_volume              100810 non-null  object
 10  AI95_volume              100810 non-null  object
 11  AI98_volume              100810 non-null  object
 12  diesel_volume            100810 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.0+ MB


In [96]:
# заменяем запятую на точку и приводим к float
for col in groc_and_fuels.columns[2:]:
    groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)

# добавляем год (чтобы усреднить за период)
groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year
groc_and_fuels.head(3)

C:\Users\013\AppData\Local\Temp/ipykernel_2988/1479746939.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)
C:\Users\013\AppData\Local\Temp/ipykernel_2988/1479746939.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year


,region_code,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume,observ_year
0,1,01.01.2019,5023.51,4301.36,5001.97,42.96,46.87,50.20,45.53,103778.69,56968.84,121.19,13973.49,2019
1,1,02.01.2019,5086.32,4354.00,5064.53,42.98,46.92,50.30,45.38,185820.84,119512.26,95.09,39283.90,2019
2,1,03.01.2019,4930.56,4214.30,4911.48,41.13,46.86,50.77,45.46,219913.38,126519.19,107.82,45498.67,2019


In [97]:
groc_and_fuels.observ_year.unique()

array([2019, 2020, 2021, 2022], dtype=int64)

In [98]:
# оставляем только 21, 20 и 19 гг.
groc_and_fuels = groc_and_fuels.loc[groc_and_fuels.observ_year.isin((2021, 2020, 2019))]

In [99]:
# заменяем 0 на None - т.е. не цена 0, а отсутствие наблюдений
groc_and_fuels = groc_and_fuels.replace(0, None)

In [100]:
# средняя стоимость за период по регионам
avg_groc_and_fuels = groc_and_fuels.groupby(['region_code', 'observ_year'], as_index=False)[['cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']].mean()

- разворачиваем таблицу - чтобы получить столбцы по годам
- плюс 33 столбца

In [101]:
val_columns = ['cons_basket_working_age', 'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 
               'AI95_cost', "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
               "AI98_volume", 'diesel_volume']

In [102]:
pivot_groc_and_fuels =  pd.pivot_table(avg_groc_and_fuels, values=val_columns, 
               index='region_code', columns=['observ_year'], aggfunc='mean').reset_index()

In [103]:
pivot_groc_and_fuels.head()

region_code  AI92_cost                         AI92_volume  \
observ_year                   2019       2020       2021          2019   
0                     1  43.332219  43.547923  44.553863  2.211384e+05   
1                     2  41.124986  42.006940  43.900411  2.018062e+06   
2                     3  42.091425  43.820874  45.771123  4.270480e+05   
3                     4  42.244000  41.944891  45.373288  8.576184e+04   
4                     5  42.700493  42.862760  45.421041  1.113027e+04   

                                         AI95_cost                        ...  \
observ_year          2020          2021       2019       2020       2021  ...   
0            1.874788e+05  2.081576e+05  48.051562  48.533852  50.663260  ...   
1            1.795260e+06  1.718155e+06  44.246466  45.193770  47.296274  ...   
2            2.976584e+05  3.115107e+05  43.764466  45.792158  48.114411  ...   
3            6.974820e+04  9.066889e+04  44.202932  44.186038  47.365479  ...   
4            7.508397e+03  2.065066e+04  45.790740  46.983880  50.277397  ...   

            cons_basket_pens cons_basket_working_age               \
observ_year             2021                    2019         2020   
0                5031.818411             5005.002466  5146.416749   
1                4804.212411             4637.134110  4863.667842   
2                5167.355589             5084.651507  5433.417869   
3                4772.020712             4623.268630  4949.242541   
4                5594.561616             5007.779616  5325.073033   

                         diesel_cost                        diesel_volume  \
observ_year         2021        2019       2020       2021           2019   
0            5914.448685   45.404822  46.012650  47.691973   65073.028384   
1            5657.522795   45.621288  46.787104  48.158740  322267.884164   
2            6080.860192   47.587370  50.227650  53.012384  103348.248466   
3            5604.221205   46.775726  48.395792  50.237151   16605.699342   
4            6542.970658   42.806658  44.108743  46.466849    1374.417068   

                                           
observ_year           2020           2021  
0             60822.033087   69281.618521  
1            286174.082486  277899.671616  
2             79717.239016  103949.519370  
3             13378.189536   21544.138055  
4              2379.751612    5120.080192  

[5 rows x 34 columns]

- избавляемся от мультииндекса в названиях столбцов

In [104]:
pivot_groc_and_fuels.columns = ['region_code'] + [f'{el[0]}_{str(el[1])}' for el in pivot_groc_and_fuels.columns.values[1:]]

In [105]:
pivot_groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   region_code                   85 non-null     int64  
 1   AI92_cost_2019                85 non-null     float64
 2   AI92_cost_2020                85 non-null     float64
 3   AI92_cost_2021                85 non-null     float64
 4   AI92_volume_2019              85 non-null     float64
 5   AI92_volume_2020              85 non-null     float64
 6   AI92_volume_2021              85 non-null     float64
 7   AI95_cost_2019                85 non-null     float64
 8   AI95_cost_2020                85 non-null     float64
 9   AI95_cost_2021                85 non-null     float64
 10  AI95_volume_2019              85 non-null     float64
 11  AI95_volume_2020              85 non-null     float64
 12  AI95_volume_2021              85 non-null     float64
 13  AI98_co

- добавляем строку для кода региона 95 == 20

In [106]:
val = pivot_groc_and_fuels.loc[pivot_groc_and_fuels.region_code == 20, pivot_groc_and_fuels.columns[1:]].values[0]

In [107]:
pivot_groc_and_fuels.loc[len(pivot_groc_and_fuels.index)] = [95, *val]

In [108]:
pivot_groc_and_fuels.head(5)

,region_code,AI92_cost_2019,AI92_cost_2020,AI92_cost_2021,AI92_volume_2019,AI92_volume_2020,AI92_volume_2021,AI95_cost_2019,AI95_cost_2020,AI95_cost_2021,...,cons_basket_pens_2021,cons_basket_working_age_2019,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021
0,1.0,43.332219,43.547923,44.553863,2.211384e+05,1.874788e+05,2.081576e+05,48.051562,48.533852,50.663260,...,5031.818411,5005.002466,5146.416749,5914.448685,45.404822,46.012650,47.691973,65073.028384,60822.033087,69281.618521
1,2.0,41.124986,42.006940,43.900411,2.018062e+06,1.795260e+06,1.718155e+06,44.246466,45.193770,47.296274,...,4804.212411,4637.134110,4863.667842,5657.522795,45.621288,46.787104,48.158740,322267.884164,286174.082486,277899.671616
2,3.0,42.091425,43.820874,45.771123,4.270480e+05,2.976584e+05,3.115107e+05,43.764466,45.792158,48.114411,...,5167.355589,5084.651507,5433.417869,6080.860192,47.587370,50.227650,53.012384,103348.248466,79717.239016,103949.519370
3,4.0,42.244000,41.944891,45.373288,8.576184e+04,6.974820e+04,9.066889e+04,44.202932,44.186038,47.365479,...,4772.020712,4623.268630,4949.242541,5604.221205,46.775726,48.395792,50.237151,16605.699342,13378.189536,21544.138055
4,5.0,42.700493,42.862760,45.421041,1.113027e+04,7.508397e+03,2.065066e+04,45.790740,46.983880,50.277397,...,5594.561616,5007.779616,5325.073033,6542.970658,42.806658,44.108743,46.466849,1374.417068,2379.751612,5120.080192


- приводим типы

In [109]:
for col in pivot_groc_and_fuels.columns[1:]:
    pivot_groc_and_fuels[col] = pd.to_numeric(pivot_groc_and_fuels[col], downcast='float')
    
pivot_groc_and_fuels.region_code = pd.to_numeric(pivot_groc_and_fuels.region_code, downcast='integer')

In [110]:
pivot_groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 85
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   region_code                   86 non-null     int8   
 1   AI92_cost_2019                86 non-null     float32
 2   AI92_cost_2020                86 non-null     float32
 3   AI92_cost_2021                86 non-null     float32
 4   AI92_volume_2019              86 non-null     float32
 5   AI92_volume_2020              86 non-null     float32
 6   AI92_volume_2021              86 non-null     float32
 7   AI95_cost_2019                86 non-null     float32
 8   AI95_cost_2020                86 non-null     float32
 9   AI95_cost_2021                86 non-null     float32
 10  AI95_volume_2019              86 non-null     float32
 11  AI95_volume_2020              86 non-null     float32
 12  AI95_volume_2021              86 non-null     float32
 13  AI98_co

#### - соединяем

In [111]:
%%time
data = data.merge(pivot_groc_and_fuels, how='left', on='region_code')

Wall time: 2.68 s


In [112]:
data.shape

(1918094, 121)

#### численность по регионам - здесь немного сложнее

##### - для начала максимально заполню пропуски по городам - если нет поселения а есть регион = город

In [113]:
# для тестов
addresses_col = ['region', 'city', 'settlement']
addresses  = data[addresses_col]

- приводим к одному регистру

In [114]:
for col in addresses_col:
    addresses[col] = addresses[col].apply(lambda x: x.title() if x else x)

C:\Users\013\AppData\Local\Temp/ipykernel_2988/3677381831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses[col] = addresses[col].apply(lambda x: x.title() if x else x)


-  нет ни города ни поселения

In [115]:
addresses.loc[addresses.city.isna() & addresses.settlement.isna()].shape, addresses.loc[addresses.city.isna() & addresses.settlement.isna()].region.unique()

((484950, 3),
 array(['Московская', 'Москва', 'Санкт-Петербург', 'Ленинградская',
        'Хабаровский', 'Севастополь', 'Самарская', 'Оренбургская',
        'Костромская', 'Иркутская', 'Татарстан', 'Ульяновская',
        'Пензенская', 'Новгородская', 'Ростовская', 'Пермский',
        'Удмуртская', 'Свердловская', 'Липецкая', 'Ивановская',
        'Башкортостан', 'Новосибирская', 'Приморский', 'Саратовская',
        'Тюменская', 'Тамбовская', 'Тверская', 'Карелия', 'Забайкальский',
        'Мурманская', 'Воронежская', 'Краснодарский', 'Красноярский',
        'Белгородская', 'Ставропольский', 'Нижегородская', 'Курганская',
        'Томская', 'Тульская', 'Марий Эл', 'Астраханская', 'Калужская',
        'Смоленская', 'Владимирская', 'Камчатский', 'Коми', 'Челябинская',
        'Калининградская', 'Бурятия', 'Алтай', 'Крым', 'Кировская',
        'Сахалинская', 'Рязанская', 'Курская', 'Псковская',
        'Архангельская', 'Ханты-Мансийский Автономный Округ - Югра',
        'Чувашская Республи

- нет города, есть поселения

In [116]:
addresses.loc[addresses.city.isna() & ~addresses.settlement.isna()].region.unique()

array(['Московская', 'Москва', 'Алтайский', 'Новгородская', 'Алтай',
       'Саха /Якутия/', 'Нижегородская', 'Ненецкий', 'Архангельская',
       'Адыгея', 'Брянская', 'Приморский', 'Сахалинская', 'Кировская',
       'Ставропольский', 'Санкт-Петербург', 'Псковская', 'Ленинградская',
       'Бурятия', 'Забайкальский', 'Волгоградская', 'Краснодарский',
       'Кемеровская Область - Кузбасс', 'Новосибирская', 'Хабаровский',
       'Томская', 'Самарская', 'Смоленская', 'Свердловская', 'Чеченская',
       'Ханты-Мансийский Автономный Округ - Югра', 'Воронежская',
       'Красноярский', 'Рязанская', 'Мордовия', 'Тамбовская',
       'Оренбургская', 'Пермский', 'Чукотский', 'Ярославская',
       'Костромская', 'Татарстан', 'Башкортостан',
       'Чувашская Республика -', 'Хакасия', 'Крым', 'Тульская',
       'Саратовская', 'Северная Осетия - Алания', 'Пензенская',
       'Владимирская', 'Севастополь', 'Ульяновская', 'Калужская',
       'Омская', 'Вологодская', 'Челябинская', 'Ростовская',
    

- ни региона, ни города ни поселения

In [117]:
# ну, хоть что-то, да есть практически для каждой строки
addresses.loc[addresses.city.isna() & addresses.settlement.isna() & addresses.region.isna()]

,region,city,settlement
1882141,None,None,None


- для тех, у кого нет города, но в графе регион - название города (это для крупных городов, где город имеет свой код региона) - заполняем поле city

        'Москва'
        'Санкт-Петербург'
        'Севастополь'

In [118]:
# это интересно - у них всех код региона 50 - московская область
# те с адресом всё нетривиально
data.loc[data.region == 'БАЙКОНУР']

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,cons_basket_pens_2021,cons_basket_working_age_2019,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021
639112,9901033939,"ООО ""СКАЗКА""",468320,БАЙКОНУР,None,None,47.25,12300,ACTIVE,2008-12-16,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639113,9901034629,"ООО ""ПРОФИ-СОФТ""",468321,БАЙКОНУР,None,None,62.0,12300,ACTIVE,2012-01-23,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639114,9901035171,"ООО ""ВЕНТА""",468320,БАЙКОНУР,None,None,41.20,12300,ACTIVE,2014-01-30,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639115,9901035647,"ООО ""РС №2""",468320,БАЙКОНУР,None,None,43.11,12300,ACTIVE,2015-03-12,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639116,9901034636,"ООО ""ОРЕХ""",468320,БАЙКОНУР,None,None,38.32.2,12300,ACTIVE,2012-02-13,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762447,9901001535,"ООО ""ФЕРМЕРСКОЕ ХОЗЯЙСТВО БОЛАШАК""",468325,БАЙКОНУР,None,None,68.20,12300,ACTIVE,1996-03-19,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
762448,9901001888,ПК ГСК № 24,468320,БАЙКОНУР,None,None,94.99,20101,ACTIVE,1996-04-22,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
762449,9901000940,"ООО ""ТЕХНОСЕРВИС""",468325,БАЙКОНУР,None,None,47.30,12300,ACTIVE,1996-11-29,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
762450,9901001165,"ООО ""БУТЯ-БАЙКОНУР""",468320,БАЙКОНУР,None,None,47.75,12300,ACTIVE,1996-03-01,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25


In [119]:
data.loc[data.region == 'БАЙКОНУР'].region_code.unique()

array([50], dtype=int8)

- т.е. можно заполнить почти все пропуски города (328 300 из 334 976)

In [120]:
# условие для заполнения
cond = addresses.city.isna() & addresses.region.isin(('Москва', 'Санкт-Петербург', 'Севастополь'))

In [121]:
addresses.city = addresses.city.where(~cond, addresses.region)

C:\Users\013\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [122]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1918094 entries, 0 to 1918093
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   region      object
 1   city        object
 2   settlement  object
dtypes: object(3)
memory usage: 58.5+ MB


In [123]:
# осталось 13 тыс с только областью - неплохо
addresses.loc[addresses.city.isna() & addresses.settlement.isna()]

,region,city,settlement
7,Московская,None,None
10,Московская,None,None
14,Московская,None,None
16,Московская,None,None
17,Московская,None,None
...,...,...,...
1916075,Псковская,None,None
1916106,Псковская,None,None
1916587,Удмуртская,None,None
1917981,Московская,None,None


##### - переходим к основным данным
    - приводим к одному виду
    - заполняем город, для кого можно

In [124]:
for col in addresses_col:
    data[col] = data[col].apply(lambda x: x.title() if x else x)

In [125]:
data[addresses_col].head()

,region,city,settlement
0,Волгоградская,Волгоград,None
1,Волгоградская,Волгоград,None
2,Волгоградская,Волгоград,None
3,Волгоградская,Волгоград,None
4,Волгоградская,Волгоград,None


In [126]:
cond = data.city.isna() & data.region.isin(('Москва', 'Санкт-Петербург', 'Севастополь'))

In [127]:
data.city = data.city.where(~cond, data.region)

In [128]:
data[addresses_col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1918094 entries, 0 to 1918093
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   region      object
 1   city        object
 2   settlement  object
dtypes: object(3)
memory usage: 58.5+ MB


In [129]:
data.loc[data.region == 'Москва', addresses_col].head(3)

,region,city,settlement
15,Москва,Москва,None
19,Москва,Москва,None
24,Москва,Москва,None


- заменяем некоторые названия городов

In [130]:
city_replace = {
    'Королев': 'Королёв',
    'Белохолуницкое Городское Поселение': 'Белохолуницкое',
    'Мо Сельское Поселение Следневское': 'Следневское',
    'Камень-На-Оби': 'Камень-на-Оби',
    'Широкобуеракское Мо': 'Широкобуеракское',
    'Новопушкинское Мо': 'Новопушкинское',
    'Ростов-На-Дону': 'Ростов-на-Дону',
    'Сольцы 2': 'Сольцы',
    'Буденновск': 'Будённовск',
    'Городское Поселение Снегири': 'Снегири',
    'Калач-На-Дону': 'Калач-на-Дону',
    'Славянск-На-Кубани': 'Славянск-на-Кубани',
    'Терсинское Мо': 'Терсинское',
    'Николаевск-На-Амуре': 'Николаевск-на-Амуре',
    'Комсомольск-На-Амуре': 'Комсомольск-на-Амуре',
}

In [131]:
data.city.replace(city_replace, inplace=True)

- названия settlements так вычистить не получится - их слишком много и слишком мало совпадений

##### - и только теперь переходим к данным переписи (данные за 20 год)

In [133]:
vpn_data = pd.read_excel('raw_data/tab-5_VPN-2020.xlsx', header=3)
population = vpn_data[['Unnamed: 0', 'Unnamed: 1']].rename(columns={'Unnamed: 0': 'locality', 'Unnamed: 1': 'population'})

In [134]:
population.head()

,locality,population
0,Российская Федерация,147182123
1,Городское население,110075322
2,Сельское население,37106801
3,Центральный федеральный округ,40334532
4,Городское население,33118003


- уберём пропуски

In [135]:
population.loc[population.population == '-']

,locality,population
237,Сельское население,-
443,Сельское население,-
648,Сельское население,-
922,прочие сельские населенные пункты,-
971,прочие сельские населенные пункты,-
...,...,...
27398,Сельское население,-
27510,Сельское население,-
27587,пгт Беличан,-
27605,пгт Верхний Ат-Урях,-


In [136]:
population = population.loc[population.population != '-']

- создадим словарь город-численность

In [137]:
# порядок важен - в городах сперва ищем город
# выражение не оптимально!
def get_population(name):
    val = population.loc[population['locality']\
          .str \
          .contains(rf'г?. {name}\b|пгт?. {name}\b|[Сс]ело {name}\b|пос[ёе]лок {name}\b|п. {name}\b|[Сс]ельское поселение {name}\b|^{name}\b сельское поселение|Поселение {name}\b|[Дд]еревня {name}\b|город {name}\b')]\
          .population.values
    return val[0] if val.any() else None

In [138]:
cities = data.city.unique()

In [139]:
cities, len(cities)

(array(['Волгоград', None, 'Одинцово', ...,
        'Верхошижемское Городское Поселение', 'Акбулак', 'Пыталово'],
       dtype=object),
 1299)

In [140]:
%%time
city_to_population = {key: get_population(key) for key in cities}

Wall time: 2min 23s


In [141]:
city_to_population['Курган']

13991

- то же самое для settlememt, но гораздо дольше (так как уникальных около 20 тыс) и гораздо менее эффективно - будет всего около 4 тыс значений
- остальные придётся заполнять (например, медианой всех значений менее 3 тыс) - именно для таких населённых пунктов нет данных

In [146]:
def get_population_settl(name):
    val = population.loc[population['locality']\
          .str \
          .contains(rf'пгт?. {name}\b|[Сс]ело {name}\b|пос[ёе]лок {name}\b|п?. {name}\b|[Сс]ельское поселение {name}\b|^{name}\b сельское поселение|Поселение {name}\b|[Дд]еревня {name}\b')]\
          .population.values
    return val[0] if val.any() else None

In [147]:
settlements = data.settlement.unique()

Wall time: 35min 58s

In [164]:
%%time
settlement_to_population = {key: get_population_settl(key) for key in settlements}

C:\Users\013\AppData\Local\Temp/ipykernel_2988/913981156.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  val = population.loc[population['locality']\


Wall time: 38min 21s


In [ ]:
# with open('parsed_data/settlement_to_population.pickle', 'wb') as f:
#     pickle.dump(settlement_to_population, f)

In [148]:
# # просто прочитаю из файла
# with open('parsed_data/settlement_to_population.pickle', 'rb') as f:
#     settlement_to_population = pickle.load(f)

- словарь с численностью для регионов - самый полный и чистый

In [149]:
# словарь перевода кода региона в имя региона
# вообще мб путаница: разные версии кодов - для каких-то регионов несколько кодов, некоторые 
# коды объединяются, используются временно... переименовываются...
region_code_to_name = {
#    0: 'Unknown',  # неизвестный регион
    1: 'Республика Адыгея',
    2: 'Республика Башкортостан',
    3: 'Республика Бурятия',
    4: 'Республика Алтай',
    5: 'Республика Дагестан',
    6: 'Республика Ингушетия',
    7: 'Кабардино-Балкарская Республика',
    8: 'Республика Калмыкия',
    9: 'Карачаево-Черкесская Республика',
    10: 'Республика Карелия',
    11: 'Республика Коми',
    12: 'Республика Марий Эл',
    13: 'Республика Мордовия',
    14: 'Республика Саха (Якутия)',
    15: 'Республика Северная Осетия-Алания',
    16: 'Республика Татарстан',
    17: 'Республика Тыва',
    18: 'Удмуртская Республика',
    19: 'Республика Хакасия',
    20: 'Чеченская Республика',
    21: 'Чувашская Республика',
    22: 'Алтайский край',
    23: 'Краснодарский край',
    24: 'Красноярский край',
    25: 'Приморский край',
    26: 'Ставропольский край',
    27: 'Хабаровский край',
    28: 'Амурская область',
    29: 'Архангельская область',
    30: 'Астраханская область',
    31: 'Белгородская область',
    32: 'Брянская область',
    33: 'Владимирская область',
    34: 'Волгоградская область',
    35: 'Вологодская область',
    36: 'Воронежская область',
    37: 'Ивановская область',
    38: 'Иркутская область',
    39: 'Калининградская область',
    40: 'Калужская область',
    41: 'Камчатский край',  # Камчатская область
    42: 'Кемеровская область – Кузбасс',
    43: 'Кировская область',
    44: 'Костромская область',
    45: 'Курганская область',
    46: 'Курская область',
    47: 'Ленинградская область',
    48: 'Липецкая область',
    49: 'Магаданская область',
    50: 'Московская область',
    51: 'Мурманская область',
    52: 'Нижегородская область',
    53: 'Новгородская область',
    54: 'Новосибирская область',
    55: 'Омская область',
    56: 'Оренбургская область',
    57: 'Орловская область',
    58: 'Пензенская область',
    59: 'Пермский край',  # Пермская область
    60: 'Псковская область',
    61: 'Ростовская область',
    62: 'Рязанская область',
    63: 'Самарская область',
    64: 'Саратовская область',
    65: 'Сахалинская область',
    66: 'Свердловская область',
    67: 'Смоленская область',
    68: 'Тамбовская область',
    69: 'Тверская область',
    70: 'Томская область',
    71: 'Тульская область',
    72: 'Тюменская область',
    73: 'Ульяновская область',
    74: 'Челябинская область',
    75: 'Забайкальский край',  # Читинская область
    76: 'Ярославская область',
    77: 'г. Москва - городское население',  # так в таблице, ищу точное совпадение
    78: 'г. Санкт-Петербург - городское население',
    79: 'Еврейская автономная область',
    83: 'Ненецкий автономный округ',
    86: 'Ханты-Мансийский автономный округ - Югра',
    87: 'Чукотский автономный округ',
    89: 'Ямало-Ненецкий автономный округ',
    90: 'Московская область',  # то же, что и код 50
    91: 'Республика Крым',
    92: 'г. Севастополь',
    93: 'Краснодарский край',  # == 23 Краснодарский край
#    94: #  == 0 - Байконур, данных нет
    95: 'Чеченская Республика'#  == 20 Чеченская респ.
}

In [150]:
np.sort(data.region_code.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 86, 87, 89, 90, 91,
       92, 95], dtype=int8)

In [151]:
# словарик код - численность
get_p = lambda x: population.loc[population.locality == region_code_to_name[x], 'population'].values[0]
region_code_to_population = {key: get_p(key) for key in region_code_to_name}

In [152]:
region_code_to_population[1]

496934

#### - объединяем

In [153]:
data['region_population'] = data.region_code.replace(region_code_to_population)

In [154]:
data.loc[data.region_code == 1].head(3)

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,cons_basket_working_age_2019,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021,region_population
2239,0105018478,ООО ФИРМА ОШТЕН-1,385000,Адыгея,Майкоп,None,43.91,12300,ACTIVE,2001-06-07,...,5005.002441,5146.416992,5914.44873,45.404823,46.01265,47.691971,65073.027344,60822.03125,69281.617188,496934
2240,0105024440,"ООО ""КПЦ""",385019,Адыгея,Майкоп,None,96.09,12300,ACTIVE,1996-08-20,...,5005.002441,5146.416992,5914.44873,45.404823,46.01265,47.691971,65073.027344,60822.03125,69281.617188,496934
2241,0105027730,"ООО ФИРМА ""ЦЕНТР""",385009,Адыгея,Майкоп,None,77.39,12300,ACTIVE,1998-03-26,...,5005.002441,5146.416992,5914.44873,45.404823,46.01265,47.691971,65073.027344,60822.03125,69281.617188,496934


In [155]:
data['city_population'] = data.city.replace(city_to_population)

In [156]:
data.loc[data.city == 'Королёв'].head(3)

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021,region_population,city_population
1637,7703819290,"ООО ""КОМПАНИЯ ""МОСКОВСКИЕ КАНИКУЛЫ""",141075,Московская,Королёв,None,79.11,12300,ACTIVE,2014-10-09,...,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25,8524665,228095.0
9180,7701679369,"ООО ""КОНЬЯЧНЫЙ ДОМ ""АРМКО""",141080,Московская,Королёв,None,46.34.2,12300,ACTIVE,2006-09-13,...,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25,8524665,228095.0
11611,5638077275,"ПК ""АЛЬТЕРНА""",141065,Московская,Королёв,None,64.99,20100,ACTIVE,2020-08-20,...,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25,8524665,228095.0


In [165]:
%%time
data['settlement_population'] = data.settlement.replace(settlement_to_population)

Wall time: 6min 58s


- приводим типы

In [158]:
data.region_population = pd.to_numeric(data.region_population, downcast='integer')

In [166]:
for col in ['city_population', 'settlement_population']:
    data[col] = pd.to_numeric(data[col], downcast='float')

- итоговый датасет

In [167]:
data.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1918094 entries, 0 to 1918093
Data columns (total 124 columns):
 #    Column                        Non-Null Count    Dtype         
---   ------                        --------------    -----         
 0    inn                           1918094 non-null  object        
 1    shortName                     1918094 non-null  object        
 2    index                         1911937 non-null  object        
 3    region                        1918039 non-null  object        
 4    city                          1748836 non-null  object        
 5    settlement                    197561 non-null   object        
 6    okved2                        1916562 non-null  object        
 7    okopf                         1918094 non-null  object        
 8    statusCode                    1918094 non-null  object        
 9    registrationDate              1918093 non-null  datetime64[ns]
 10   authorizedCapital             1838094 non-null  floa

In [ ]:
# with open('parsed_data/data.pickle', 'wb') as f:
#     pickle.dump(data, f)

- больше 100 столбцов
- не все из них одинаково полезны
- достаточно много уйдёт после выбора цели и периода + после объединения/удаления/изменения сильно cкоррелированных признаков...